# Get them lyrics... 🥳

In this notebook, I used the Genius API to get the lyrics for the Spotify playlists I generated in the previous notebooks. I was inspired by these two GitHub repositories from <a href="https://github.com/johnwmillr/LyricsGenius2"> Johnwmillr</a> and <a href="https://github.com/MerkLerner/genius-scraper/blob/master/genius.py"> MarkLerner</a> to create the code and the functions in this notebook and also relied on the Genuis API <a href="https://docs.genius.com/#/getting-started-h1"> documentation</a>. 

For some of the songs used in the datasets, Genius did not have the lyrics and I had to look manually for the values in different websites. Also, for other songs, Genius returned the translated lyrics and not the original lyrics so there is a lot of data cleaning as well in this notebook. 

In [34]:
# Importing the relevant libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import os
import importlib
import sys

import re
import requests
from bs4 import BeautifulSoup
import lyricsgenius
from langdetect import detect
from textblob import TextBlob

In [35]:
# Setting the environmental variable 
os.environ['PYTHONIOENCODING'] = 'utf-8'

In [36]:
# Creating API access 
base_url = 'http://api.genius.com'
genius = lyricsgenius.Genius("8G8lnTddAfkzbpMqBDqx6C36cev9zm_5v2h20YaE6vgXmxFcr_IuegZTfJ1n9NpG")
genius.remove_section_headers=True
genius.verbose = False

In [8]:
# Loading Global file 
global_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\Global Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(global_path))
globalx = pd.read_csv(os.path.basename(global_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [10]:
# Inspecting the data 
globalx = globalx[['song_title', 'artist/s', 'country']]
globalx['lyrics'] = globalx['song_title']
globalx.head()

,song_title,artist/s,country,lyrics
0,Blinding Lights,The Weeknd,Global,Blinding Lights
1,Don't Start Now,Dua Lipa,Global,Don't Start Now
2,The Box,Roddy Ricch,Global,The Box
3,Dance Monkey,Tones And I,Global,Dance Monkey
4,Roses - Imanbek Remix,"['SAINt JHN', 'Imanbek']",Global,Roses - Imanbek Remix


In [37]:
# Defining the get_lyrics lyrics function 
def get_lyrics_reboot(df): 
    songs = []
    for i in range(len(df.song_title)): 
        songs.append(genius.search_song(df['song_title'][i], df['artist/s'][i]))
        for song in songs:
            if song != None: 
                df['lyrics'][i] = song.lyrics
            else: 
                df['lyrics'][i] = np.nan
    return df

In [110]:
# Running the get_lyrics function for Global lyrics 
get_lyrics_reboot(globalx)

,song_title,artist/s,country,lyrics
0,Blinding Lights,The Weeknd,Global,Yeah\n\nI've been tryna call\nI've been on my ...
1,Don't Start Now,Dua Lipa,Global,"If you don't wanna see me\n\nDid a full 180, c..."
2,The Box,Roddy Ricch,Global,Pullin' out the coupe at the lot\nTold 'em fuc...
3,Dance Monkey,Tones And I,Global,"They say, ""Oh my God, I see the way you shine\..."
4,Roses - Imanbek Remix,"['SAINt JHN', 'Imanbek']",Global,Roses\nI walked in the corner with the body sc...
5,death bed (coffee for your head) (feat. beabad...,"['Powfu', 'beabadoobee']",Global,"Don't stay awake for too long, don't go to bed..."
6,Break My Heart,Dua Lipa,Global,I've always been the one to say the first good...
7,Say So,Doja Cat,Global,"Day to night to morning, keep with me in the m..."
8,Safaera,"['Bad Bunny', 'Jowell & Randy', 'Nengo Flow']",Global,"Bla, bla, bla, bla, bla, bla\nEy, yo', yo', yo..."
9,Physical,Dua Lipa,Global,Common love isn't for us\nWe created something...


In [111]:
# Fixing missing values
globalx.lyrics[23] = """(Wheezy outta here)
I'm a late bloomer, third year on the scene
Twenty racks don't feel like nothin' to me
Came from the streets, it turned me to a beast
Invisible set, diamonds huggin' my piece
Book me for a show, I need eighty at least
I want the smoke, ain't no keepin' the peace
Keep me a razor when I'm in the East
Open 'em up just like a surgery
Everything burnin' around me, I'm lit
Show a lil' attitude, swap out the bitch
I spent two-fifty, don't know where it went
My hood on my back, I gotta represent
Toronto, you useless, you don't got a pole
Warm that boy up, he got shot in the cold
Thirty rounds in the clip, let it unload
I fuck the bitch I picked right out of Vogue
Took twenty bitches on my first vacay
I ain't pickin' up, I'm in Turks, lil' baby
Every other watch got diamonds in the face
Pullin' out, ask to take a taste, lil' baby
Pullin' out fifty racks, walkin' out of Chase
If I drop dead, I'll be hard to replace
Hardly get thirsty, got water like a lake
My brother got locked, another bond I gotta pay
Trappin' on the block, I'd never serve a cop
I always kept a Glock, I ain't 'bout to see the grave ('Bout to see the grave)
Pull up, I'll make a quick stop (Yeah)
I shop one stop, I'm high for seven days (High for seven days)
Who is that lil' nigga icy hoppin' out a Wraith? (Hoppin' out a Wraith?)
Chopper get it choppin' like a blade (Chop)
You ain't get no money but you poppin' on your page (Poppin' on your page)
Trust me, we could never be the same (Never be the same)
You know I like the syrup, low stress in Turks (Yeah)
I'm puttin' in work, I beat her out the frame (Beat her out the frame, Wheezy outta here)
She sweet, I was movin' off a Perc'
I bought her a Birk' and a birthday cake (Birthday cake)
Line his ass up like a barber do the fade (Barber do the fade)
Fed your ho crêpes and grapes
Cash stuffed away, this a billion on the way (Billion on the way)
We just split a billi' with the gang
Took twenty bitches on my first vacay
I ain't pickin' up, I'm in Turks, lil' baby
Every other watch got diamonds in the face
Pullin' out, ask to take a taste, lil' baby
Pullin' out fifty racks, walkin' out of Chase
If I drop dead, I'll be hard to replace
Hardly get thirsty, got water like a lake
My brother got locked, another bond I gotta pay
First time in Turks, had a whole bunch of work
Had to bring two jets, had a whole lot of babes (Yeah)
I first take a game-changin' verse
Swear the change gon' hurt, but it really ain't a game (Yeah)
Shawty like to twerk, I bring the rage (Yeah)
Really get to trippin' on the stage
Never ever let 'em see the man unless they pay (Alright)
Niggas havin' beef, I hope they pray
Cactus Jack me up and down, head to toe
Make sure the ones you with on go
Make sure the ones you with is with it
Make sure the ones you with, ah (Yeah)
She need a quick tan, come to Turks with the G's (Yeah)
Work for that body, need more than degrees
Fillin' her nose with some shit I can ski with
Say this her first time, it's hard to believe
Took twenty bitches on my first vacay
I ain't pickin' up, I'm in Turks, lil' baby
Every other watch got diamonds in the face
Pullin' out, ask to take a taste, lil' baby
Pullin' out fifty racks, walkin' out of Chase
If I drop dead, I'll be hard to replace
Hardly get thirsty, got water like a lake
My brother got locked, another bond I gotta pay"""

globalx.lyrics[25] = """Yeh, yeh Todavía yo te quiero
Pero sé que e' un error
Porque ya tú no me quieres
Y sin ti me va mejor
Y si veo a tu mamá (Ey)
Yo le pregunto por ti
Pa' ver si ya tienes a alguien (¿Uh?)
Alguien que te haga feliz
Y aquí estoy arrebata'o
Pensando en to'a las vece' que te lo metí
Pensando en to'a las vece' que estuve pa' ti (Pa' ti, pa' ti, pa' ti)
No sé por qué diablo' me engaño (No)
Diciendo que te olvidé cuando te extraño
Solo comparto memes, ya yo no escribo nada (Damn)
Y no he borra'o tu foto, solo la puse privada
Maldito año nuevo y lo que me trajo, ey
Me botaron del trabajo por estar mirando pa' abajo
Pensando en ti siempre cabizbajo
Me veo gordito, nada que rebajo (Nah)
No sé por qué la vida me ultrajo (No)
Pensando coger un atajo
Conocí a alguien, pero no sé, nunca encajo (Ah)
Al meno' que sea' tú (Tú, tú, tú, tú)
Baby, te quiero aunque diga lo contrario
Llevo sei' mese' solitario, pero
Hoy salí con lo' muchacho' a beber (Jaja)
Y dije que de ti no iba hablar (No, no, no)
Son las 5, ya va amanecer (Ku-ku-ku-ku)
Si no prenden, la voy a llamar
Hoy salí con lo' muchacho' a beber (Jaja)
Y dije que de ti no iba hablar (No, no, no)
Son las 5, ya va amanecer (Ku-ku-ku-ku)
Si no prenden, la voy a llamar
Todavía yo te quiero (Yo te quiero)
Pero sé que e' un error (Se hace, no)
Porque ya tú no me quieres (No, no)
Y sin ti me va mejor (Juju)
Y si veo a tu mamá (Mami), ey
Yo le pregunto por ti (Okey)
Pa' ver si ya tienes a alguien
Alguien que te haga feliz
Jajaja"""

globalx.lyrics[35] = """Workin' on a weekend like usual
Way off in the deep end like usual
Niggas swear they passed us, they doin' too much
Haven't done my taxes, I'm too turnt up
Virgil got a Patek on my wrist goin' nuts
Niggas caught me slippin' once, okay, so what?
Someone hit your block up, I'd tell you if it was us
Manor house in Rosewood, this shit too plush
Say my days are numbered, but I keep wakin' up
Know you see my texts, baby, please say some
Wine by the glass, your man a cheapskate, huh?
Niggas gotta move off my release day, huh
Bitch, this is fame, not clout
I don't even know what that's about, watch your mouth
Baby got a ego twice the size of the crib
I can never tell her shit, it is what it is (What)
Said what I had to and did what I did (Ayy)
Never turn my back on FBG, God forbid
Virgil got the Patek on my wrist doin' front flips
Givin' you my number, but don't hit me on no dumb shit
Workin' on a weekend like usual
Way off in the deep end like usual (Like usual)
Niggas swear they passed us, they doin' too much
Haven't done my taxes, I'm too turnt up
Virgil got a Patek on my wrist goin' nuts
Niggas caught me slippin' once, okay, so what?
Someone hit your block up, I'd tell you if it was us
Manor house in Rosewood, this shit too plush
It's cool, man, got red bottoms on
Life is good, you know what I mean? Like
Yeah, hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Granny, she was standin' right there while I catch a play on a brick, woo
I make them lil' niggas go haywire, Taliban in this bitch, woo
I done been down bad in them trenches, had to ride with that stick, woo
Who gave you pills? Who gave you that dust? Pluto sent you on a lick, woo
Too many convicts that roll with me to play in this shit, woo
I'm tryna avoid nonsense, get Osama spray in this bitch, woo
They at the candlelight lightin' it up, nigga, anybody can get it, woo
I'm on a PJ lightin' it up, Backwood full of sticky, woo
I'm tryna tote that Draco in London and it's extended, woo
They gotta stretch a nigga out, we gon' die for this shit, woo
Yeah, I ride for my niggas, I lie to my bitch, woo
We some poor, high-class niggas, made it, we rich, yeah
I was at the bando, got a penthouse for a closet, woo
It's like a chandelier on my neck, my wrist, woo
I got pink toes that talk different languages, woo
Got Promethazine in my blood and Percocet, yeah
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Racks by the ton, I call up Serena
I go tremendo for new fettuccine
All fact though, carat the pinky
All fact though, we ordered the Fiji
I'm in the loop with the voo, I'm in the loop with the woo, which one you workin'?
I'll put your face on the news, I'll put the pussy on the shirt after I murk it
Then make 'em go shoot up the hearse, cost me a quarter bird, nigga, it's worth it
And you a maniac, a fuckin' alien, how you splurgin'?
Got that kitty cat, I'm havin' fun with that, goin' Birkin
Yeah, hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, yeah
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, uh
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch"""

In [38]:
# Defining a text cleaning function to get rid of unwanted values
def clean_lyrics(lyrics): 
    lyrics = re.sub('\n', ' ', lyrics)
    lyrics = re.sub('\'', '', lyrics)
    lyrics = re.sub('\u2005', ' ', lyrics)
    lyrics = re.sub('\u2212', ' ', lyrics)   # '\u010d'
    lyrics = re.sub('\u200a', '', lyrics)
    lyrics = re.sub('\u200b', '', lyrics)
    lyrics = re.sub('\u0107', '', lyrics)
    lyrics = re.sub('\u010d', '', lyrics)
    lyrics = re.sub('\u0131', '', lyrics)
    lyrics = re.sub('\u015f', '', lyrics)
    lyrics = re.sub('\u205f', ' ', lyrics)
    lyrics = re.sub('-', ' ', lyrics)
    lyrics = re.sub('nigga', 'n****', lyrics)
    lyrics = lyrics.lower()
    return lyrics

In [293]:
# Running the clean lyrics function on Global lyrics
globalx.lyrics = globalx.lyrics.apply(lambda x: clean_lyrics(x))
globalx.lyrics[24]

'ante tú me pichaba (tú me pichaba) ahora yo picheo (hmm, nah) antes tú no quería (no quería) ahora yo no quiero (hmm, no) ante tú me pichaba ( chaba) ahora yo picheo (jaja) antes tú no quería (ey) ahora yo no quiero no, tranqui  yo perreo sola (hmm, ey) yo perreo sola (perreo sola; jaja; hmm hmm) yo perreo sola (jaja, hmm; ey) yo perreo sola (perreo sola) okay, okay, ey, ey, ey  que ningún baboso se le pegue (no) la disco se prende cuando ella llegue (¡wuh!) a los hombres los tiene de hobby una malcriá como nairobi (jaja) y tú la ve bebiendo de la botella (ey) los nene y las nena quieren con ella tiene má de veinte, me enseño la cédula (ajá) ey, del amor e una incrédula (¡wuh!) ella está soltera antes que se pusiera de moda (ey) no cree en amor desde "amorfoda" (no) el dj la pone y se las sabe todas se trepa en la mesa y que se jodan (wuh) en el perreo no se quita (¡no!) fuma y se pone bellaquita te llama si te necesita pero por ahora está solita  ella perrea sola (wuh) ey, ey, ey, ey

In [39]:
# Defining a function to wirte the lyrics to file 
def write_to_file(df): 
    path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\Lyrics'
    os.chdir(path)
    with open('{} Lyrics.txt'.format(df.country[8]), 'w') as lyrics_file: 
        for item in df.lyrics: 
            lyrics_file.write(item)
    #lyrics_file = '{} lyrics.txt'.format(df.country[8])
        lyrics_file.close()
    return 'Done!'

In [106]:
# Writing the lyrics to file with the new function 
write_to_file(globalx)

'Done!'

In [40]:
# Load the USA dataset
usa_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\United States Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(usa_path))
usa = pd.read_csv(os.path.basename(usa_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [41]:
# Inspect the USA dataset 
usa = usa[['song_title', 'artist/s', 'country']]
usa['lyrics'] = usa['song_title']
usa.head()

,song_title,artist/s,country,lyrics
0,Blinding Lights,The Weeknd,United States,Blinding Lights
1,The Box,Roddy Ricch,United States,The Box
2,Turks (feat. Travis Scott),"['NAV', 'Gunna', 'Travis Scott']",United States,Turks (feat. Travis Scott)
3,Blueberry Faygo,Lil Mosey,United States,Blueberry Faygo
4,Don't Start Now,Dua Lipa,United States,Don't Start Now


In [42]:
# Get lyrics for the USA
get_lyrics_reboot(usa)

,song_title,artist/s,country,lyrics
0,Blinding Lights,The Weeknd,United States,Yeah\n\nI've been tryna call\nI've been on my ...
1,The Box,Roddy Ricch,United States,Pullin' out the coupe at the lot\nTold 'em fuc...
2,Turks (feat. Travis Scott),"['NAV', 'Gunna', 'Travis Scott']",United States,"Wheezy outta here\n\nGeç çiçek açan biriyim, ü..."
3,Blueberry Faygo,Lil Mosey,United States,"Damn, Callan (Damn, Callan)\n\nOne bad bitch a..."
4,Don't Start Now,Dua Lipa,United States,"If you don't wanna see me\n\nDid a full 180, c..."
5,Say So,Doja Cat,United States,"Day to night to morning, keep with me in the m..."
6,WHATS POPPIN,Jack Harlow,United States,"What's poppin'? (Pooh, you a fool for this one..."
7,In Your Eyes,The Weeknd,United States,I just pretend that I'm in the dark\nI don't r...
8,Savage,Megan Thee Stallion,United States,"I'm that bitch (Yeah)\nBeen that bitch, still ..."
9,Intentions (feat. Quavo),"['Justin Bieber', 'Quavo']",United States,"Imagen perfecta, no necesitas filtro\nMagnífic..."


In [43]:
# Fixing missing values 
usa.lyrics[2] = """(Wheezy outta here)
I'm a late bloomer, third year on the scene
Twenty racks don't feel like nothin' to me
Came from the streets, it turned me to a beast
Invisible set, diamonds huggin' my piece
Book me for a show, I need eighty at least
I want the smoke, ain't no keepin' the peace
Keep me a razor when I'm in the East
Open 'em up just like a surgery
Everything burnin' around me, I'm lit
Show a lil' attitude, swap out the bitch
I spent two-fifty, don't know where it went
My hood on my back, I gotta represent
Toronto, you useless, you don't got a pole
Warm that boy up, he got shot in the cold
Thirty rounds in the clip, let it unload
I fuck the bitch I picked right out of Vogue
Took twenty bitches on my first vacay
I ain't pickin' up, I'm in Turks, lil' baby
Every other watch got diamonds in the face
Pullin' out, ask to take a taste, lil' baby
Pullin' out fifty racks, walkin' out of Chase
If I drop dead, I'll be hard to replace
Hardly get thirsty, got water like a lake
My brother got locked, another bond I gotta pay
Trappin' on the block, I'd never serve a cop
I always kept a Glock, I ain't 'bout to see the grave ('Bout to see the grave)
Pull up, I'll make a quick stop (Yeah)
I shop one stop, I'm high for seven days (High for seven days)
Who is that lil' nigga icy hoppin' out a Wraith? (Hoppin' out a Wraith?)
Chopper get it choppin' like a blade (Chop)
You ain't get no money but you poppin' on your page (Poppin' on your page)
Trust me, we could never be the same (Never be the same)
You know I like the syrup, low stress in Turks (Yeah)
I'm puttin' in work, I beat her out the frame (Beat her out the frame, Wheezy outta here)
She sweet, I was movin' off a Perc'
I bought her a Birk' and a birthday cake (Birthday cake)
Line his ass up like a barber do the fade (Barber do the fade)
Fed your ho crêpes and grapes
Cash stuffed away, this a billion on the way (Billion on the way)
We just split a billi' with the gang
Took twenty bitches on my first vacay
I ain't pickin' up, I'm in Turks, lil' baby
Every other watch got diamonds in the face
Pullin' out, ask to take a taste, lil' baby
Pullin' out fifty racks, walkin' out of Chase
If I drop dead, I'll be hard to replace
Hardly get thirsty, got water like a lake
My brother got locked, another bond I gotta pay
First time in Turks, had a whole bunch of work
Had to bring two jets, had a whole lot of babes (Yeah)
I first take a game-changin' verse
Swear the change gon' hurt, but it really ain't a game (Yeah)
Shawty like to twerk, I bring the rage (Yeah)
Really get to trippin' on the stage
Never ever let 'em see the man unless they pay (Alright)
Niggas havin' beef, I hope they pray
Cactus Jack me up and down, head to toe
Make sure the ones you with on go
Make sure the ones you with is with it
Make sure the ones you with, ah (Yeah)
She need a quick tan, come to Turks with the G's (Yeah)
Work for that body, need more than degrees
Fillin' her nose with some shit I can ski with
Say this her first time, it's hard to believe
Took twenty bitches on my first vacay
I ain't pickin' up, I'm in Turks, lil' baby
Every other watch got diamonds in the face
Pullin' out, ask to take a taste, lil' baby
Pullin' out fifty racks, walkin' out of Chase
If I drop dead, I'll be hard to replace
Hardly get thirsty, got water like a lake
My brother got locked, another bond I gotta pay"""

usa.lyrics[9] = """Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Shout-out to your mom and dad for makin' you
Standin' ovation, they did a great job raisin' you
When I create, you're my muse
That kind of smile that makes the news
Can't nobody throw shade on your name in these streets
Triple threat, you a boss, you a bae, you a beast
You make it easy to choose
You got a mean touch, I can't refuse (No, I can't refuse it)
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Already pass, you don't need no approval
Good everywhere, don't worry 'bout no refusal
Second to none, you got the upper hand now
Don't need a sponsor, nope, you're the brand now
You're my rock, my Colorado
Get that ring, just like Toronto
Love you now, a little more tomorrow
This how I feel, act like you know that you are
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread (Whip it)
Heart full of equity, you're an asset (Asset)
Make sure that you don't need no mentions (Yeah, yeah)
Yeah, these are my only intentions (Quavo)
No cap, no pretendin', you don't need mentions (No cap)
Got 'em sayin' "goals," they don't wanna be independent ('Pendent)
Tell them to mind your business (Woo), we in our feelings
It's fifty-fifty percentage (Fifty), attention, we need commitment (Oh)
We gotta both admit it (Both), it's funny, we both listen (Both)
It's a blessing (Blessing) 'cause we both get it (Both)
You the best thing (Woo), and I don't need a witness (Best thing)
I'ma find me a ring and pray it's perfect fitted (Perfect, perfect)
Picture perfect, you don't need no filter (No filter)
Gorgeous, make 'em drop dead, you a killer (Oh-oh)
Shower you with all my attention (I will)
Yeah, these are my only intentions (Yeah)
Stay in the kitchen cookin' up, got your own bread (You do)
Heart full of equity, you're an asset (Uh-huh)
Make sure that you don't need no mentions (No mentions)
Yeah, these are my only intentions
Only intentions
That's all I plan to do"""

usa.lyrics[15] = """Mustard on the beat, ho
Shawty made that ass clap, she don't need no applause
High fashion like Goyard
G-Wagen or the Rover?
I put some ice on you 'cause you got a cold heart
I know I gotta keep my shawty on go, go, go
Drop that ass to the floor-floor, yeah
Ahh, woah
You ain't gotta deal with none of these niggas no more
If we hop in the Benz is that okay?
Is it okay if I call you my Prada bae?
I ain't no player, I just got a lot of baes
But let me tell you, I like you a lot, bae
I wanna start at the top and the bottom, bae
Know you want the shoe with the red at the bottom, bae
You know I like when you ride at the top, bae
She want Chane'-ne' and Dio'-o'
I'm only doin' cash out, I don't need promo
I pull up to the high rise, I'm in the four-four
Inside cocoa
If I got a feeling, I keep it inside my heart
And I keep a Patek, too, 'cause I don't do facades
You see my diamonds even when I'm in the dark
And since you got it, it make you go and do anything you want to
Shawty made that ass clap, she don't need no applause (No applause)
High fashion like Goyard (Like Goyard)
G-Wagen or the Rover? (Skrrt)
I put some ice on you 'cause you got a cold heart (Ooh-ooh)
I know I gotta keep my shawty on go, go, go (Shawty on go, go, yeah)
Drop that ass to the floor-floor, yeah
Ahh, woah
You ain't gotta deal with none of these niggas no more
If we hop in the Benz is that okay?
Is it okay if I call you my Prada bae?
I ain't no player, I just got a lot of baes
But let me tell you, I like you a lot, bae
I wanna start at the top and the bottom, bae
Know you want the shoe with the red at the bottom, bae
You know I like when you ride at the top, bae
If I hit it from the back, she gon' hit the sidewalk
If she got a best friend with her, take her back to my loft
Got her five in the mornin', wildin', wildin'
And her name ain't Megan but she a stallion, stallion
Never keep my hoes divided, 'member I was pullin' up in the valley
And you know I take her soul when she ridin'
Sweater and her sneakers all Kenzo
Shawty know to drop it low like a limbo every time we kick it like Timbo, woah, woah, woah
Shawty made that ass clap, she don't need no applause (Nah, nah)
I put some ice on you 'cause you got a cold heart (Ooh-ooh)
I know I gotta keep my shawty on go, go, go (Yeah)
Drop that ass to the floor-floor, yeah
Shawty made that ass clap, she don't need no applause
High fashion like Goyard
G-Wagen or the Rover?
I put some ice on you 'cause you got a cold heart
I know I gotta keep my shawty on go, go, go
Drop that ass to the floor-floor, yeah
Ahh, woah
You ain't gotta deal with none of these niggas no more
If we hop in the Benz is that okay?
Is it okay if I call you my Prada bae?
I ain't no player, I just got a lot of baes
But let me tell you, I like you a lot, bae
I wanna start at the top and the bottom, bae
Know you want the shoe with the red at the bottom, bae
You know I like when you ride at the top, bae"""

usa.lyrics[17] = """Workin' on a weekend like usual
Way off in the deep end like usual
Niggas swear they passed us, they doin' too much
Haven't done my taxes, I'm too turnt up
Virgil got a Patek on my wrist goin' nuts
Niggas caught me slippin' once, okay, so what?
Someone hit your block up, I'd tell you if it was us
Manor house in Rosewood, this shit too plush
Say my days are numbered, but I keep wakin' up
Know you see my texts, baby, please say some
Wine by the glass, your man a cheapskate, huh?
Niggas gotta move off my release day, huh
Bitch, this is fame, not clout
I don't even know what that's about, watch your mouth
Baby got a ego twice the size of the crib
I can never tell her shit, it is what it is (What)
Said what I had to and did what I did (Ayy)
Never turn my back on FBG, God forbid
Virgil got the Patek on my wrist doin' front flips
Givin' you my number, but don't hit me on no dumb shit
Workin' on a weekend like usual
Way off in the deep end like usual (Like usual)
Niggas swear they passed us, they doin' too much
Haven't done my taxes, I'm too turnt up
Virgil got a Patek on my wrist goin' nuts
Niggas caught me slippin' once, okay, so what?
Someone hit your block up, I'd tell you if it was us
Manor house in Rosewood, this shit too plush
It's cool, man, got red bottoms on
Life is good, you know what I mean? Like
Yeah, hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Granny, she was standin' right there while I catch a play on a brick, woo
I make them lil' niggas go haywire, Taliban in this bitch, woo
I done been down bad in them trenches, had to ride with that stick, woo
Who gave you pills? Who gave you that dust? Pluto sent you on a lick, woo
Too many convicts that roll with me to play in this shit, woo
I'm tryna avoid nonsense, get Osama spray in this bitch, woo
They at the candlelight lightin' it up, nigga, anybody can get it, woo
I'm on a PJ lightin' it up, Backwood full of sticky, woo
I'm tryna tote that Draco in London and it's extended, woo
They gotta stretch a nigga out, we gon' die for this shit, woo
Yeah, I ride for my niggas, I lie to my bitch, woo
We some poor, high-class niggas, made it, we rich, yeah
I was at the bando, got a penthouse for a closet, woo
It's like a chandelier on my neck, my wrist, woo
I got pink toes that talk different languages, woo
Got Promethazine in my blood and Percocet, yeah
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Racks by the ton, I call up Serena
I go tremendo for new fettuccine
All fact though, carat the pinky
All fact though, we ordered the Fiji
I'm in the loop with the voo, I'm in the loop with the woo, which one you workin'?
I'll put your face on the news, I'll put the pussy on the shirt after I murk it
Then make 'em go shoot up the hearse, cost me a quarter bird, nigga, it's worth it
And you a maniac, a fuckin' alien, how you splurgin'?
Got that kitty cat, I'm havin' fun with that, goin' Birkin
Yeah, hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, yeah
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, uh
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch"""

usa.lyrics[26] = """I want it that way
I want it that way
I want it that way
I want it that way
But I don't wanna go out bad, wanna go out sad, wanna go out that way (No)
I'm with the winnin' team, they make sure I'm not in last place (Let's go)
If I wake up, don't make no money, that's a sad day (Woah)
Twenty-five hundred on my shirt what the tag say (Let's go)
I want it that way
I want it that way
I want it that way, woah
I want it that way
Woah, she from Atlanta, she went to Cascade, ayy (Let's get it)
She ride the D like she's tryna drag race (Woah)
Don't slow me right down, I like it fast-paced, ayy
That is enough about that girl, everyone know it is my world
If he dropped an album, thought it was trash day (Yeah, woah)
I'm on my own, not talkin' masturbate, yeah (Ew)
She gave me dome 'til I graduate, woah
I'ma grab a Bentley, Mean might go and grab a Wraith, yeah
I had to snap back into reality and go grab a fitted (Fitted)
My jeans, yeah, they fitted (Woo)
But Lil Uzi, he is so far from the timid (Let's go)
The reason I'm rich
'Cause I had to go and just fix my percentage (Let's go)
You know that I'm winnin'
My white girl got black card and it got no limit (Woo)
I don't wanna go out bad, don't wanna go out sad, not that way (Yeah)
I'm with the winnin' team, they make sure I'm not in last place (Yes)
If I wake up, don't make no money, that's a sad day (Woah)
Twenty-five hundred for my shirt what the tag say (Yeah)
And I want it that way
I want it that way
I want it that way, woah
I want it that way
When I'm in DC, make the hoes go-go (Go)
Yes, I'm slimy like a snail, but I'm no slowpoke (Slow)
Only reason I didn't kick her out 'cause she gon' deepthroat (Yeah)
Hit it no protection, injection, yeah, this shit is lethal (Go)
They laugh at me because I'm emo (Yeah)
I killed my girlfriend, that's why I'm single (Fuck her)
Can't call 911 'cause I'm in Reno (Yeah)
Gave away my thumbs and I stood three toes (Let's go, let's go)
Diamonds hittin', bling, bling (Let's go)
Make the chopper sing, sing (Let's go, let's go, let's go)
He a killer (What?), same thing (Same thing)
Eyes low, Simmons, Ming (Yeah)
I don't wanna go out bad, don't wanna go out sad, not that way
I'm with the winnin' team, they make sure I'm not in last place (Yes)
If I wake up, don't make no money, that's a sad day (Why? Woah)
Twenty-five hundred for my shirt what the tag say (Yeah)
And I want it that way
I want it that way
I want it that way, woah
I want it that way
I want it that way
I want it that way
I want it that way
I want it that way"""

usa.lyrics[34] = """I ain't mad at 'em, baby
If I seen't you out, me too would be up on you crazy
Diamond in the rough, you look as good as Oprah's bank account
I just wanna take you out
Fuck you in your mama house
Overseas, I fly you out
Is it trickin' if she really love me? Let's just find it out
Baby, what's up with your mouth?
City girl straight from the South
Back home on a presi', I'm Obama, ayy, I condone the drama, ayy
I tongued down Madonna, ayy, do you need pajamas? Yeah
Stayin' the night, girl, you promised it
I hide in the cave like Osama did
I'm blowin' a bag in the Diamond District
You need me like dollars that are owed to you, yeah
Money and me are the same, but I just don't fold for you, yeah
Run me a body, I'll put a Range on the road for you
I don't know what you were told, but I ain't mad at you, baby
If I seen't you out, me too would be up on you crazy
Diamond in the rough, you look as good as Oprah's bank account
I just wanna take you out
Fuck you in your mama house
Overseas, I fly you out
Is it trickin' if she really love me? Let's just find it out
Baby, what's up with your mouth?
City girl straight from the South
Is they mad that you fuck with me? (Is they mad, mad, mad?)
Is they mad that you ride around in a Bentley? (Skrrt)
Is they mad that you not with them? (Is they mad?)
Same hoes hatin' be in my DM (That's too sad)
Lockin' it down, lockin' it down, lockin' it down, I want you (I want you)
I want them to know you're my baby boo (My boo)
We still make it lit when ain't nothin' to do
Every time we step out, niggas look at you
(No, no, no, no, make him hit it and then quit it)
I ain't mad at 'em, baby
If I seen't you out, me too would be up on you crazy
Diamond in the rough, you look as good as Oprah's bank account
I just wanna take you out
You look as good as Oprah's bank account
I just wanna take you out
You look as good as Oprah's bank account
I just wanna take you out
You look as good as Oprah's bank account
I ain't mad at 'em, baby
My bitch pull up lookin' like Oprah bank (Let's go)
Take a pic with a bitch and she faint (Yeah)
She look like a goddess, but she ain't no saint
My bitch 'bout it 'bout it, she need her a tank (Uh)
You know ain't no limit on blue hundreds
My bitch like a big bag of money, this new money (Chill)
You gon' make the bitch think that you love her (Fuck you)
Hold her hand right in front you, we too public (Aw)
Take the lil' nigga bitch, he ain't do nothin' (Yeah)
Now his bitch goin' Baby on Baby (Let's go)
She want me to stay, I ain't stayin'
And I hope that these niggas don't play, I ain't playin' (Let's go)
Your bitch lookin' good as Oprah bank account
I'm the type to take a nigga ho from him and take her out
Keep the butt, I'll take the mouth
(No, no, no, no, make him hit it and then quit it)
I ain't mad at 'em, baby (Let's go)
If I seen't you out, me too would be up on you crazy (Let's go)
Diamond in the rough, you look as good as Oprah's bank account
I just wanna take you out
Fuck you in your mama house
Overseas, I fly you out
Is it trickin' if she really love me? Let's just find it out
Baby, what's up with your mouth?
City girl straight from the South
I just wanna take you out
You look as good as Oprah's bank account
I just wanna take you out
You look as good as Oprah's bank account
I just wanna take you out
You look as good as Oprah's bank account
I ain't mad at 'em, baby"""

usa.lyrics[42] = """It's funny, 
the shit I put on this song ain't gon' sound nothin' like the shit we was just doin'
Oh, yeah
I just want bad bitches
Baddest, them bitches is the baddest, uh, uh
When I take the molly, I'm a savage
Uh, I say, uh
I don't need no molly to be savage, uh
When I'm on that molly, I feel savage, uh, uh
She the definition of a bad bitch
Stole her, I'm the definition of a bandit, uh, ayy
I don't need no molly to be savage, uh, ayy
But when I'm on the molly, I feel savage
Ayy, my girl the definition of a bad bitch
Stole her heart, I'm the definition of a bandit
Put the Percs down and picked up the jiggas, jiggas, jiggas
Tommy in the fucking Tommy Hilfiger, 'figer, 'figer
That Tommy hit a nigga, Tommy Hilfiger, fuck niggas
I'm nice, when I'm high off the pills, I'ma fuck with her
I don't smoke skunk, but tonight I'm getting stuck, nigga
Pour the codeine up and put some molly in the cup with it
I know she a freak, uh-huh, she gon' fuck with it
She my velcro, uh-huh, guess I'm stuck with her
I dive in it like a sailor, I love to nail her
Addicted to her paraphernalia, I had to tell her
I see it like a fortune teller
Your ex-nigga did good, I could do better
Bad bitch from the woods, I think she a hunter
She a killer and an eater, she a Jeffrey Dahmer
I can tell when she in her feelings, I can read her like a book
No TEC, no Beretta, FN on me, am I understood?
Yeah, yeah, yeah
I don't need no molly to be savage, uh
When I'm on that molly, I feel savage, uh, uh
She the definition of a bad bitch
Stole her, I'm the definition of a bandit, uh, ayy
I don't need no molly to be savage, uh, ayy
But when I'm on the molly, I feel savage
Ayy, my girl the definition of a bad bitch
Stole her heart, I'm the definition of a bandit
My brother point her out and she a bad bitch, I'm on her
Must ain't heard that I'm a savage, once I get a bitch, I own her
I see she got swag, I got cash so I want her
See this four-five in my pants, put on your ass, push up on bruh
Shawty, she a rider with that glizzy on her (Glizzy on her)
And shawty, I'ma dada with no semi on me (Semi on me)
If we got a problem, we get rid of homie (Yeah)
Put twenty thousand in your pocket, we gon' get the money (Yeah)
I'm the definition of a bandit (Honest)
Took your heart from out his hands and still ain't saying shit (On my mama)
Some new killers in my circle you done ran with
Like this dirty .38, this bitch'll damage
Popping wheelies, 4K Trey, call when you land with it (Pop, pop, pop)
I let you drive and slide with my bros where they be laying with it (Skrrt)
I work this bitch, I open up a can with it
Like fuck the stove, I make it jump without my hand in it
I don't need no molly to be savage, uh
When I'm on that molly, I feel savage, uh, uh
She the definition of a bad bitch
Stole her, I'm the definition of a bandit, uh, ayy
I don't need no molly to be savage, uh, ayy
But when I'm on the molly, I feel savage
Ayy, my girl the definition of a bad bitch
Stole her heart, I'm the definition of a bandit"""

In [44]:
# Applying the cleaning function to the USA lyrics 
usa.lyrics = usa.lyrics.apply(lambda x: clean_lyrics(x))
usa.lyrics[12]

'ive always been the one to say the first goodbye had to love and lose a hundred million times had to get it wrong to know just what i like now im falling you say my name like i have never heard before im indecisive, but, this time, i know for sure i hope im not the only one that feels it all are you falling?  centre of attention you know you can get whatever you want from me whenever you want it, baby its you in my reflection im afraid of all the things it could do to me if i wouldve known it, baby  i wouldve stayed at home cause i was doing better alone but when you said, "hello" i knew that was the end of it all i shouldve stayed at home cause now there aint no letting you go am i falling in love with the one that could break my heart? oh no, i was doing better alone but when you said, "hello" i knew that was the end of it all i shouldve stayed at home cause now there aint no letting you go am i falling in love with the one that could break my heart?  i wonder, when you go, if i sta

In [ ]:
# Writing the lyrics to a file
write_to_file(usa)

In [222]:
# Loading the file for the UK
uk_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\United Kingdom Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(uk_path))
uk = pd.read_csv(os.path.basename(uk_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [224]:
# Inspecting the dataset 
uk = uk[['song_title', 'artist/s', 'country']]
uk['lyrics'] = uk['song_title']
uk.head()

,song_title,artist/s,country,lyrics
0,Blinding Lights,The Weeknd,United Kingdom,Blinding Lights
1,Roses - Imanbek Remix,"['SAINt JHN', 'Imanbek']",United Kingdom,Roses - Imanbek Remix
2,Physical,Dua Lipa,United Kingdom,Physical
3,Don't Start Now,Dua Lipa,United Kingdom,Don't Start Now
4,The Box,Roddy Ricch,United Kingdom,The Box


In [225]:
# Running the get lyrics function for the UK chart 
get_lyrics_reboot(uk)

,song_title,artist/s,country,lyrics
0,Blinding Lights,The Weeknd,United Kingdom,Yeah\n\nI've been tryna call\nI've been on my ...
1,Roses - Imanbek Remix,"['SAINt JHN', 'Imanbek']",United Kingdom,Roses\nI walked in the corner with the body sc...
2,Physical,Dua Lipa,United Kingdom,Common love isn't for us\nWe created something...
3,Don't Start Now,Dua Lipa,United Kingdom,"If you don't wanna see me\n\nDid a full 180, c..."
4,The Box,Roddy Ricch,United Kingdom,Pullin' out the coupe at the lot\nTold 'em fuc...
5,Break My Heart,Dua Lipa,United Kingdom,I've always been the one to say the first good...
6,Say So,Doja Cat,United Kingdom,"Day to night to morning, keep with me in the m..."
7,Blueberry Faygo,Lil Mosey,United Kingdom,"Damn, Callan (Damn, Callan)\n\nOne bad bitch a..."
8,Intentions (feat. Quavo),"['Justin Bieber', 'Quavo']",United Kingdom,"Imagen perfecta, no necesitas filtro\nMagnífic..."
9,Dance Monkey,Tones And I,United Kingdom,"They say, ""Oh my God, I see the way you shine\..."


In [231]:
# Fixing missing values 
uk.lyrics[8] = """Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Shout-out to your mom and dad for makin' you
Standin' ovation, they did a great job raisin' you
When I create, you're my muse
That kind of smile that makes the news
Can't nobody throw shade on your name in these streets
Triple threat, you a boss, you a bae, you a beast
You make it easy to choose
You got a mean touch, I can't refuse (No, I can't refuse it)
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Already pass, you don't need no approval
Good everywhere, don't worry 'bout no refusal
Second to none, you got the upper hand now
Don't need a sponsor, nope, you're the brand now
You're my rock, my Colorado
Get that ring, just like Toronto
Love you now, a little more tomorrow
This how I feel, act like you know that you are
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread (Whip it)
Heart full of equity, you're an asset (Asset)
Make sure that you don't need no mentions (Yeah, yeah)
Yeah, these are my only intentions (Quavo)
No cap, no pretendin', you don't need mentions (No cap)
Got 'em sayin' "goals," they don't wanna be independent ('Pendent)
Tell them to mind your business (Woo), we in our feelings
It's fifty-fifty percentage (Fifty), attention, we need commitment (Oh)
We gotta both admit it (Both), it's funny, we both listen (Both)
It's a blessing (Blessing) 'cause we both get it (Both)
You the best thing (Woo), and I don't need a witness (Best thing)
I'ma find me a ring and pray it's perfect fitted (Perfect, perfect)
Picture perfect, you don't need no filter (No filter)
Gorgeous, make 'em drop dead, you a killer (Oh-oh)
Shower you with all my attention (I will)
Yeah, these are my only intentions (Yeah)
Stay in the kitchen cookin' up, got your own bread (You do)
Heart full of equity, you're an asset (Uh-huh)
Make sure that you don't need no mentions (No mentions)
Yeah, these are my only intentions
Only intentions
That's all I plan to do"""

uk.lyrics[29] = """Workin' on a weekend like usual
Way off in the deep end like usual
Niggas swear they passed us, they doin' too much
Haven't done my taxes, I'm too turnt up
Virgil got a Patek on my wrist goin' nuts
Niggas caught me slippin' once, okay, so what?
Someone hit your block up, I'd tell you if it was us
Manor house in Rosewood, this shit too plush
Say my days are numbered, but I keep wakin' up
Know you see my texts, baby, please say some
Wine by the glass, your man a cheapskate, huh?
Niggas gotta move off my release day, huh
Bitch, this is fame, not clout
I don't even know what that's about, watch your mouth
Baby got a ego twice the size of the crib
I can never tell her shit, it is what it is (What)
Said what I had to and did what I did (Ayy)
Never turn my back on FBG, God forbid
Virgil got the Patek on my wrist doin' front flips
Givin' you my number, but don't hit me on no dumb shit
Workin' on a weekend like usual
Way off in the deep end like usual (Like usual)
Niggas swear they passed us, they doin' too much
Haven't done my taxes, I'm too turnt up
Virgil got a Patek on my wrist goin' nuts
Niggas caught me slippin' once, okay, so what?
Someone hit your block up, I'd tell you if it was us
Manor house in Rosewood, this shit too plush
It's cool, man, got red bottoms on
Life is good, you know what I mean? Like
Yeah, hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Granny, she was standin' right there while I catch a play on a brick, woo
I make them lil' niggas go haywire, Taliban in this bitch, woo
I done been down bad in them trenches, had to ride with that stick, woo
Who gave you pills? Who gave you that dust? Pluto sent you on a lick, woo
Too many convicts that roll with me to play in this shit, woo
I'm tryna avoid nonsense, get Osama spray in this bitch, woo
They at the candlelight lightin' it up, nigga, anybody can get it, woo
I'm on a PJ lightin' it up, Backwood full of sticky, woo
I'm tryna tote that Draco in London and it's extended, woo
They gotta stretch a nigga out, we gon' die for this shit, woo
Yeah, I ride for my niggas, I lie to my bitch, woo
We some poor, high-class niggas, made it, we rich, yeah
I was at the bando, got a penthouse for a closet, woo
It's like a chandelier on my neck, my wrist, woo
I got pink toes that talk different languages, woo
Got Promethazine in my blood and Percocet, yeah
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Racks by the ton, I call up Serena
I go tremendo for new fettuccine
All fact though, carat the pinky
All fact though, we ordered the Fiji
I'm in the loop with the voo, I'm in the loop with the woo, which one you workin'?
I'll put your face on the news, I'll put the pussy on the shirt after I murk it
Then make 'em go shoot up the hearse, cost me a quarter bird, nigga, it's worth it
And you a maniac, a fuckin' alien, how you splurgin'?
Got that kitty cat, I'm havin' fun with that, goin' Birkin
Yeah, hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, woo
I done flew one out to Spain to be in my domain and Audemars-ed the bitch, woo
Dropped three dollars on a ring, cost a Bentley truck, lil' bitch, woo
I was in the trap servin' cocaine, I ain't been the same since, woo
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, yeah
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch, uh
Hunnid thousand for the cheapest ring on a nigga finger, lil' bitch"""

uk.lyrics[37] = "flowers"

uk.lyrics[46] = """They say he likes a good time (My oh my)
He comes alive at midnight (Every night)
My mama doesn't trust him (My oh my)
He's only here for one thing, but (So am I)
Yeah, a little bit older, a black leather jacket
A bad reputation, insatiable habits
He was onto me, one look and I couldn't breathe, yeah
I said, if he kissed me, I might let it happen (Oh)
I swear on my life that I've been a good girl (Oh)
Tonight, I don't wanna be her
They say he likes a good time (My oh my)
He comes alive at midnight (Every night)
My mama doesn't trust him (My oh my)
He's only here for one thing, but (Let's go, so am I)
Look, I'm the type to make her turn on her daddy (Oh yeah)
DaBaby make her forget what she learned from her daddy
I don't be tripping on lil' shawty, I let her do whatever she please
I don't be kissing on lil' shawty, she don't be kissing on me either
She came with you, then left with me
I went up a point, let's call it even (Yeah, yeah)
Don't like the car she in, gon' end up buyin' her a new Bimmer (Let's go)
That girl know what she want, she make me take it off when she see me (Let's go)
She say I make her wet whenever my face pop up on TV
I had to say, "No disrespect, gotta do it safe or you can keep it"
Pop star, I'm fresh up out the trap and I'm going Bieber
She know I'm a call away, she can drop a pin and I'd come meet her
Stand next to me, you gon' end up catchin' a fever (Yeah, yeah)
I'm hot
I swear on my life that I've been a good girl (Good girl, good girl)
Tonight, I don't wanna be her
They say he likes a good time (My oh my)
He comes alive at midnight (Every time)
(He comes alive, oh every night)
My mama doesn't trust him (My oh my)
He's only here for one thing, but (So am I)
My, my, my, my, my oh my
My mama doesn't trust you, baby
My, my, my, my, my oh my
And my daddy doesn't know you, no
My, my, my, my, my oh my (Oh my, my, my, my, my)
My, my, my, my, my oh my (My, my, my, ooh)
They say he likes a good time (My oh my)
He comes alive at midnight (Every night)
My mama doesn't trust him (My oh my)
He's only here for one thing, but (So am I)"""

In [268]:
# Running the cleaning function on the UK lyrics 
uk.lyrics = uk.lyrics.apply(lambda x: clean_lyrics(x))
uk.lyrics[0]

'yeah  ive been tryna call ive been on my own for long enough maybe you can show me how to love, maybe im going through withdrawals you dont even have to do too much you can turn me on with just a touch, baby  i look around and sin citys cold and empty (oh) no ones around to judge me (oh) i cant see clearly when youre gone  i said, ooh, im blinded by the lights no, i cant sleep until i feel your touch i said, ooh, im drowning in the night oh, when im like this, youre the one i trust hey, hey, hey  im running out of time cause i can see the sun light up the sky so i hit the road in overdrive, baby, oh  the citys cold and empty (oh) no ones around to judge me (oh) i cant see clearly when youre gone  i said, ooh, im blinded by the lights no, i cant sleep until i feel your touch i said, ooh, im drowning in the night oh, when im like this, youre the one i trust  im just coming back to let you know (back to let you know) i could never say it on the phone (say it on the phone) will never let 

In [300]:
# Writing the lyrics to file 
write_to_file(uk)

'Done!'

In [297]:
# Importing the Brazil dataset 
brazil_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\Brazil Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(brazil_path))
brazil = pd.read_csv(os.path.basename(brazil_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [298]:
# Inspecting the Brazil dataset 
brazil = brazil[['song_title', 'artist/s', 'country']]
brazil['lyrics'] = brazil['song_title']
brazil.head()

,song_title,artist/s,country,lyrics
0,Liberdade Provisória - Ao Vivo,Henrique & Juliano,Brazil,Liberdade Provisória - Ao Vivo
1,A Gente Fez Amor - Ao Vivo,Gusttavo Lima,Brazil,A Gente Fez Amor - Ao Vivo
2,BRABA,Luísa Sonza,Brazil,BRABA
3,Don't Start Now,Dua Lipa,Brazil,Don't Start Now
4,Graveto - Ao Vivo,Marília Mendonça,Brazil,Graveto - Ao Vivo


In [299]:
# Running the get lyrics function on Brazil 
get_lyrics_reboot(brazil)

,song_title,artist/s,country,lyrics
0,Liberdade Provisória - Ao Vivo,Henrique & Juliano,Brazil,No inicio foi assim\nTerminou tá terminado\nCa...
1,A Gente Fez Amor - Ao Vivo,Gusttavo Lima,Brazil,NaN
2,BRABA,Luísa Sonza,Brazil,"Sonza, hmm-hmm\n\nDeixa eu te falar\nEu sou do..."
3,Don't Start Now,Dua Lipa,Brazil,"If you don't wanna see me\n\nDid a full 180, c..."
4,Graveto - Ao Vivo,Marília Mendonça,Brazil,NaN
5,Litrão - Ao Vivo,Matheus & Kauan,Brazil,NaN
6,Rave de Favela (feat. BEAM),"['MC Lan', 'Major Lazer', 'Anitta', 'BEAM']",Brazil,"Ai!\nHmm\nAnitta, Major Lazer\nE ele\nMC Lan, ..."
7,S de Saudade,"['Luíza & Maurílio', 'Zé Neto & Cristiano']",Brazil,"É, olha eu aqui, na vida de solteiro que eu se..."
8,SENTADÃO,"['Pedro Sampaio', 'Felipe Original', 'JS o Mão...",Brazil,NaN
9,Bebi Minha Bicicleta (Coração Falido) - Ao Vivo,Zé Neto & Cristiano,Brazil,"Vai ter azar igual a esse caboclo eu nunca vi,..."


In [351]:
# Fixing missing values
brazil.lyrics[1] = """A gente tinha combinado
Que já tava tudo errado
Que não dava mais
Marcamos o ponto final
Mas o final é sempre igual
Você me faz voltar atrás
Enche de perfume no corpo
E um sorriso que me deixa louco
Com a intenção de provocar
Como é que eu largo dela?
E a gente fez amor, oh oh
Era pra terminar, quem disse que eu consigo?
E a gente fez amor, oh oh
Como se fosse o último pedido
E a gente fez amor, oh oh
Era pra terminar, quem disse que eu consigo?
E a gente fez amor, oh oh
Como se fosse o último pedido
Enche de perfume no corpo
E um sorriso que me deixa louco
Com a intenção de provocar
Como é que eu largo dela?
Enche de perfume no corpo
E um sorriso que me deixa louco
Com a intenção de provocar
Como é que eu largo dela?
E a gente fez amor, oh oh
Era pra terminar, quem disse que eu consigo?
E a gente fez amor, oh oh
Como se fosse o último pedido
E a gente fez amor, oh oh
Era pra terminar, quem disse que eu consigo?
E a gente fez amor, oh oh
Como se fosse o último pedido
Enche de perfume no corpo
E um sorriso que me deixa louco
Com a intenção de provocar
E como é que eu largo dela?
E a gente fez amor, oh oh
Era pra terminar, quem disse que eu consigo?
E a gente fez amor, oh oh
Como se fosse o último pedido
A gente fez amor, oh oh
Era pra terminar, quem disse que eu consigo?
E a gente fez amor, oh oh
Como se fosse o último pedido
Enche de perfume no corpo
E um sorriso que me deixa louco
Com a intenção de provocar
E como é que eu largo dela?"""

brazil.lyrics[4] = """Vou ser sincero com você
Acho que pra mim já deu
Faz um tempinho que não sou seu
Até a cama percebeu
Que esfriou demais
E o seu toque não traz
Não adianta pôr graveto
Na fogueira que não pega mais
Não pega mais, não pega mais
Você virou saudade aqui dentro de casa
Se eu te chamo pro colchão você foge pra sala
E nem se importa mais saber do que eu sinto
Poucos metros quadrados virou um labirinto
Você virou saudade aqui dentro de casa
Se eu te chamo pro colchão você foge pra sala
E nem se importa mais saber do que eu sinto
Poucos metros quadrados virou um labirinto
Vou ser sincero com você
Acho que pra mim já deu
Faz um tempinho que não sou seu
Até a cama percebeu
Que esfriou demais
E o seu toque não traz
Não adianta pôr graveto
Na fogueira que não pega mais
Não pega mais, não pega mais
Você virou saudade aqui dentro de casa
Se eu te chamo pro colchão você foge pra sala
E nem se importa mais saber do que eu sinto
Poucos metros quadrados virou um labirinto
Você virou saudade aqui dentro de casa
Se eu te chamo pro colchão você foge pra sala
E nem se importa mais saber do que eu sinto
Poucos metros quadrados virou um labirinto"""

brazil.lyrics[5] = """Não sou de fazer ameaça, mas seu beijo vai ter volta
Não tô querendo te apressar, mas minha vida já tá pronta
Tira uma selfie comigo, depois eu te mostro
Eu ainda vou namorar com essa moça da foto
Eu tenho certeza, vai ser de primeira
Se a gente ficar, é menos uma solteira nesse mundo
E menos um vagabundo!
Você decide, a minha boca ou a do litrão
Cê quer dançar comigo ou descer até o chão, sozinha?
Você quer ser da bagaceira ou quer ser minha?
Você decide, a minha boca ou a do litrão
Cê quer dançar comigo ou descer até o chão, sozinha?
Você quer ser da bagaceira ou quer ser minha?
Tá na sua mão
Vai escolher amanhecer na farra
Ou no meu colchão?
Eu tenho certeza, vai ser de primeira
Se a gente ficar, é menos uma solteira nesse mundo
E menos um vagabundo!
Você decide, a minha boca ou a do litrão
Cê quer dançar comigo ou descer até o chão, sozinha?
Você quer ser da bagaceira ou quer ser minha?
Você decide, a minha boca ou a do litrão
Cê quer dançar comigo ou descer até o chão, sozinha?
Você quer ser da bagaceira ou quer ser minha?
Você decide, a minha boca ou a do litrão
Cê quer dançar comigo ou descer até o chão, sozinha?
Você quer ser da bagaceira ou quer ser minha?
Tá na sua mão
Vai escolher amanhecer na farra
Ou no meu colchão?
Vai escolher amanhecer na farra
Ou no meu colchão?"""

brazil.lyrics[8] = """Instigada, essa danada
Movimenta e vem por cima
Vai sentando, re-rebolando
Vai, vai sentando, re-rebolando, vai
Instigada, essa danada
Movimenta e vem por cima
Vai sentando, re-rebolando
Vai, vai, vai, vai, vai
Sentadão, sentadão
Olha o movimento que ela faz
Jogando esse bundão no chão
Sentadão, sentadão
Olha o movimento que ela faz
Jogando o seu bundão no chão
Chão, chão, chão, chão
Jogando o seu bundão no chão
Chão, chão, chão
Sentadão, sentadão, sentadão
Batida envolvente, na malemolência
No beat, ela senta e representa
Senta, senta (sentadão)
Senta, senta, senta (sentadão)
Se-se-senta, senta, senta (sentadão)
Felipe Original
Fala tu que eu tô cansado
Pe-dro Sam-pai-o, vai
JS"""

brazil.lyrics[10] = """Quis se envolver porque tá com saudade
Quer voltar pra casa cansada, com a perna tremendo
Travou no movimento
Senta se tá com saudade
Mostra que tu tá a fim
Jogando, jogando, jogando, jogando
Jogando, jogando, jogando, jogando
Dança, seu bumbum balança
Tu desce, balança, balança, balança
Dança, seu bumbum balança
Tu desce, balança, balança, balança
Dança, seu bumbum balança
Tu desce, balança, balança, balança
Oi, vai de quatro, vem de quatro
Vai, vai, oi, vai de, vai de quatro
Oi, vai de quatro, vem de quatro
Vai, vai, oi, vai de, vai de
Ela brota no baile de favela (Chama), chama as amiguinha
FP lançou a braba (Vamo' bora, hein?), ai que maravilha
WC foi que falou que hoje rola ousadia
E foi Pedro que falou que hoje vai rolar festinha
E se tu quer, pode vir jogando pros cria
Quer ficar bem marolando? Então sobe balão noite e dia (É o trem)
Ela trampa com a tropa, e gosta dessa energia
Dança, teu bumbum balança
Pode se envolver com os cria
Dança, seu bumbum balança
Tu desce, balança, balança, balança
Dança, seu bumbum balança
Tu desce, balança, balança, balança
Dança, seu bumbum balança
Tu desce, balança, balança, balança
Oi, vai de quatro, vem de quatro
Vai, vai, oi, vai de, vai de quatro
Oi, vai de quatro, vem de quatro
Vai, vai, oi, vai de, vai de"""

brazil.lyrics[11] = """Se eu dissesse que tá tudo bem
Eu estaria mentido pra vocês
A marca de Sol da aliança no meu dedo mostra
Que ela me deixou tem menos de um mês
Na fronha ainda tem
O cheiro do shampoo
Na mente ainda tem ela usando aquele baby doll azul
Por isso, essa gelada eu vou beber
Em homenagem à saudade que eu tô da minha cheirosa
Que jurou na minha cara que a gente não tem mais volta
Essa gelada eu vou beber
Em homenagem à saudade que eu tô da minha cheirosa
Se ela não voltar pra mim, esse mundão pode acabar agora
E se eu dissesse que tá tudo bem
Eu estaria mentido pra vocês
A marca de Sol da aliança no meu dedo mostra
Que ela me deixou tem menos de um mês
Na fronha ainda tem
O cheiro do shampoo
Na mente ainda tem ela usando aquele baby doll azul
Por isso, essa gelada eu vou beber
Em homenagem à saudade que eu tô da minha cheirosa
Que jurou na minha cara que a gente não tem mais volta
Essa gelada eu vou beber
Em homenagem à saudade que eu tô da minha cheirosa
Se ela não voltar pra mim, esse mundão pode acabar agora
Essa gelada eu vou beber
Em homenagem à saudade que eu tô da minha cheirosa
Que jurou na minha cara que a gente não tem mais volta
Essa gelada eu vou beber
Em homenagem à saudade que eu tô da minha cheirosa
Se ela não voltar pra mim, esse mundão pode acabar agora"""

brazil.lyrics[12] = """Eu tô rindo, mas é desespero
Tô mais perdido que cego em tiroteio
Rodando a cidade
Com medo de voltar pra casa
Olha que fase
No passageiro umas garrafa
E a saudade
Tô enrolando na rua
Bebendo uma por uma
Madrugada com o vidro fechado
Chorando com tudo que toca no rádio
Parei no sinaleiro
Pra esperar o sinal vermelho
Três batidas no meu vidro
Quando vi era um bandido
Falou, perdeu
Eu disse, eu perdi mesmo
Mas foi o grande amor da minha vida
Três batidas no meu vidro
Quando vi era um bandido
Falou, perdeu
Eu disse, eu perdi mesmo
Mas foi o grande amor da minha vida
Só leva o carro e deixa as bebida
Tô enrolando na rua
Bebendo uma por uma
Madrugada com o vidro fechado
Chorando com tudo que toca no rádio
Parei no sinaleiro
Pra esperar o sinal vermelho
Três batidas no meu vidro
Quando vi era um bandido
Falou, perdeu
Eu disse, eu perdi mesmo
Mas foi o grande amor da minha vida
Três batidas no meu vidro
Quando vi era um bandido
Falou, perdeu
Eu disse, eu perdi mesmo
Mas foi o grande amor da minha vida
Só leva o carro e deixa as bebida"""

brazil.lyrics[13] = """Marca pra nós se ver, pode ser pra ferver
Hoje eu quero você, vem me satisfazer
No teu jeito que eu me amarro de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
Te avisei, não se envolve no meu papo
Não passou de um boato pra você enlouquecer
Eu te dei prazer, tu pediu prazer
Agora o que eu posso fazer? Eu não quero mais você
Quem falou que eu quero ser tua mulher?
Toma vergonha na cara e vê se larga do meu pé
Muito louca na onda do boldim
Chamei os cria pra base, vai rolar chuva de lean
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
Marca pra nós se ver, pode ser pra ferver
Hoje eu quero você, vem me satisfazer
No teu jeito que eu me amarro de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
Te avisei, não se envolve no meu papo
Não passou de um boato pra você enlouquecer
Eu te dei prazer, tu pediu prazer
Agora o que eu posso fazer? Eu não quero mais você
Quem falou que eu quero ser tua mulher?
Toma vergonha na cara e vê se larga do meu pé
Muito louca na onda do boldim
Chamei os cria pra base, vai rolar chuva de lean
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo
De quatro eu jogo o rabo, de quatro eu jogo o rabo
Sequência de toma toma, sequência de vapo vapo"""

brazil.lyrics[16] = """Se você não me disser a verdade
Não tem como eu ajudar, minha metade
Se tem outro em meu lugar
Se você quer colocar
Te deixo livre pra viver
Não prometo te esperar
Coração vai cicatrizar
E uma hora eu sei que vou te esquecer
Se era medo de me machucar
Já tem o meu perdão
Tá esperando o que?
Só quero que você seja feliz
Com ou sem mim
E quando der saudade e for dormir
Com ou sem mim
E quando perceber que eu não tô
E que não vai achar um novo amor
Vai ter que aceitar que é assim
Com ou sem mim
Se tem outro em meu lugar
Ou se você quer colocar
Te deixo livre, livre pra viver
Não prometo te esperar
Coração vai cicatrizar
E uma hora eu sei que vou te esquecer
Se era medo de me machucar
Tá esperando o que?
Só quero que você seja feliz
Com ou sem mim
E quando der saudade e for dormir
Com ou sem mim
E quando perceber que eu não tô
E que não vai achar um outro amor
Vai ter que aceitar que é assim
Com ou sem mim
Com ou sem mim
Com ou sem mim
Adicionar à playlist
Tamanho
Cifra
Imprimir
Corrigir"""

brazil.lyrics[24] = """Deu sexta-feira de novo
Já sei aonde isso vai dar
É dia de chopp em dobro
Sei lá se eu vou aguentar
Ficar sem beber, ficar sem ligar
Ficar sem chorar a a a a
Aí eu bebo
E fico tonta
Lembro de nada
Nem do meu nome
Esqueço tudo
Quase tudo
Só não esqueço
Seu telefone
Aí eu ligo, ligo, ligo, ligo, ligo, ligo
Não me atende e eu só bico, bico, bico"""

brazil.lyrics[25] = """Eu te conheci depois que a gente largou
Será onde é que arrumou todo esse ódio de mim?
Agora eu tô aqui
Num bar de currutela
Abrindo cerveja com o dente
Tentando te passar pra frente, quem dera
Por que você não me bloqueou
Pra eu parar de chorar em cima da tela?
Eu vou trocar meu celular num Nokia tijolão
Que só manda mensagem e faz ligação
Se eu ver mais um vídeo seu, sem eu, sendo feliz
Certeza que a minha vida vai tá por um triz
Me mata não! Essa internet virou arma na sua mão
Eu vou trocar meu celular num Nokia tijolão
Que só manda mensagem e faz ligação
Se eu ver mais um vídeo seu, sem eu, sendo feliz
Certeza que a minha vida vai tá por um triz
Me mata não! Essa internet virou arma na sua mão
Agora eu tô aqui
Num bar de currutela
Abrindo cerveja com o dente
Tentando te passar pra frente, quem dera
Por que você não me bloqueou
Pra eu parar de chorar em cima da tela?
Eu vou trocar meu celular num Nokia tijolão
Que só manda mensagem e faz ligação
Se eu ver mais um vídeo seu, sem eu, sendo feliz
Certeza que a minha vida vai tá por um triz
Me mata não! Essa internet virou arma na sua mão
Eu vou trocar meu celular num Nokia tijolão
Que só manda mensagem e faz ligação
Se eu ver mais um vídeo seu, sem eu, sendo feliz
Certeza que a minha vida vai tá por um triz
Me mata não! Essa internet virou arma na sua mão
Me mata não! Essa internet virou arma na sua mão
Adicionar à playlist
Tamanho Cifra Imprimir Corrigir"""

brazil.lyrics[33] = """Vytinho NG, tu viaja né B?
MC Bianca, e aê chavoso
Solta o hit do verão
Ela viaja nos maloca, os cara que é do corre
Que gera na favela, que pega e não se envolve
Ela é da zona sul, eu sou da zona norte
Vai rolar festinha, brota que o bonde tá forte
Vai rolar festinha, brota que o bonde tá forte
Se acionar a tropa, vai rolar resenha
Tudo no sigilo, tudo no esquema
Se pá tem balão, se pá tem balinha
Vou ficar na onda, vou perder a linha
Na onda do lança tu bota e não cansa
Na onda da bala eu sento na vara
Na onda do boldo, na onda do boldo
Fode, fode gostoso, fode, fode gostoso
Na onda do lança tu bota e não cansa
Na onda da bala eu sento na vara
Na onda do boldo, na onda do boldo
Fode, fode gostoso, fode, fode gostoso
Na onda do lança, lança, lança, lança, lança
Na onda da bala, bala, bala, bala, bala
Na onda do boldo, na onda do boldo
Fode, fode gostoso, fode, fode gostoso
Ela viaja nos maloca, os cara que é do corre
Que gera na favela, que pega e não se envolve
Ela é da zona sul, eu sou da zona norte
Vai rolar festinha, brota que o bonde tá forte
Vai rolar festinha, brota que o bonde tá forte
Se acionar a tropa, vai rolar resenha
Tudo no sigilo, tudo no esquema
Se pá tem balão, se pá tem balinha
Vou ficar na onda, vou perder a linha
Na onda do lança tu bota e não cansa
Na onda da bala eu sento na vara
Na onda do boldo, na onda do boldo
Fode, fode gostoso, fode, fode gostoso
Na onda do lança tu bota e não cansa
Na onda da bala eu sento na vara
Na onda do boldo, na onda do boldo
Fode, fode gostoso, fode, fode gostoso
Na onda do lança, lança, lança, lança, lança
Na onda da bala, bala, bala, bala, bala
Na onda do boldo, na onda do boldo
Fode, fode gostoso, fode, fode gostoso
Vytinho NG
MC Bianca
Mais uma que explodiu"""

brazil.lyrics[34] = """Vai, pra mim foi fácil, já sei como resolver
Simplificar o jeito de te esquecer
Até que foi facinho, eu tô legal assim
Ah, mas quem eu tô querendo mesmo enganar
Fica difícil e até chato de evitar
Sentir o fogo meu se misturar com o seu
Vamo levar junto essa briga lá pro quarto
Coloca ela pra dormir, e puxa outro papo
E aí, tem jeito ou não tem?
Eu e você, meu bem
Já sei o resultado
Ooh, meu lençol dobrado já tá todo bagunçado
E o seu cheiro no meu quarto
Tá de brincadeira
Já que esqueceu de tudo que aconteceu
Aproveita e faz amor comigo a noite inteira"""

brazil.lyrics[35] = "Áudio"

brazil.lyrics[36] = """Ela não sabe ainda
Mas eu vou ser o grande amor da sua vida
Só falta ela se apaixonar
Ah, essa hora vai chegar!
E na cabeça dela
Eu só sou mais um
Um esquema zé roela
Que vai levar pra cama e pular fora
Mal sabe ela o fim dessa história
Nós dois na cama, no meu quarto
Me dá só meia hora de amor
Com o relógio parado
Na cama, no meu quarto
A gente dorme se pegando
E acorda apaixonado
Na cama, no meu quarto
Me dá só meia hora de amor
Com o relógio parado
Na cama, no quarto
A gente dorme se pegando
E acorda apaixonado
Pedaços de amor pra todo lado"""

brazil.lyrics[40] = """Meu coração tá cansado de apanhar
Batendo errado por quem não quis me amar
Tá compensando mesmo é beber sozinho
Só um copo na mesa
Tô mais feliz com a minha cerveja
E sem ter hora pra voltar do bar
Eu vou seguir carreira solo
E tô trocando o repertório
E pra quem acha que eu tô sofrendo, que eu tô mal
Ouve o final
Eu vou seguir carreira solo
E tô trocando o repertório
E na minha cama o meu show segue normal
Com participação especial
Tá compensando mesmo é beber sozinho
Só um copo na mesa
Tô mais feliz com a minha cerveja
E sem ter hora pra voltar do bar
Eu vou seguir carreira solo
E tô trocando o repertório
E pra quem acha que eu tô sofrendo, que eu tô mal
Ouve o final
Eu vou seguir carreira solo
E tô trocando o repertório
E na minha cama o meu show segue normal
Com participação especial
Eu vou seguir carreira solo
E tô trocando o repertório
E pra quem acha que eu tô sofrendo, que eu tô mal
Ouve o final
Eu vou seguir carreira solo
E tô trocando o repertório
E na minha cama o meu show segue normal
Com participação especial"""

brazil.lyrics[41] = """Tá viciando outra boca, como fez com a minha
Esse frio na barriga já me pertenceu
Ele só tá te amando desse jeito
Porque ainda não te conheceu direito
Mas o fim dessa história é sempre o mesmo
Ela vai dar sorrisos durante o beijo
Você vai gostar
Vai planejar uma vida deitada em seu peito
Cê vai acreditar
Vai te amarrar de um jeito
Vai usar o mesmo beijo que usou pra te ganhar
Pra se despedir sem se explicar
Tô numa mesa com quatro cadeiras
Aqui já tem três enganados por ela
Bebendo, esperando a rasteira que cê vai levar dela
Fica tranquilo, que a quarta cadeira te espera
Tô numa mesa com quatro cadeiras
Aqui já tem três enganados por ela
Bebendo, esperando a rasteira que cê vai levar dela
Fica tranquilo, que a quarta cadeira te espera
Não adianta se iludir
Seu copo já tá cheio aqui pra te acudir
Tá viciando outra boca, como fez com a minha
Esse frio na barriga já me pertenceu
Ele só tá te amando desse jeito
Porque ainda não te conheceu direito
Mas o fim dessa história é sempre o mesmo
Ela vai dar sorrisos durante o beijo
Você vai gostar
Vai planejar a vida deitada em seu peito
Cê vai acreditar
Vai te amarrar de um jeito
Vai usar o mesmo beijo que usou pra te ganhar
Pra se despedir sem se explicar
Tô numa mesa com quatro cadeiras
Aqui já tem três enganados por ela
Bebendo, esperando a rasteira que você vai levar dela
Fica tranquilo, que a quarta cadeira te espera
Eu tô numa mesa com quatro cadeiras
Aqui já tem três enganados por ela
Bebendo, esperando a rasteira que você vai levar dela
Fica tranquilo, que a quarta cadeira te espera
Não adianta se iludir
Seu copo já tá cheio aqui pra te acudir
Não adianta se iludir
Seu copo já tá cheio aqui pra te acudir
Pra te acudir, pra te acudir"""

brazil.lyrics[44] = """Amar não tava nos meus planos
Não tava procurando, mas eu encontrei
Agora tô aqui falando
Foi o erro mais lindo que eu acertei
Você, você virou o meu mundo de ponta cabeça
Se um dia do amor duvidei, hoje eu tenho certeza
Tiro certo bem na mosca
Eu nem mirei quando acertei sua boca
Tiro certo bem na mosca
Era pra ser só uma noite, eu acertei, iê
Tiro certo bem na mosca
Eu nem mirei quando acertei sua boca
Tiro certo bem na mosca
Era pra ser só uma noite, eu acertei, iê
Ai ai ai, ai ai ai
Era pra ser só uma noite, eu acertei foi uma vida toda
Ai ai ai, ai ai ai
Era pra ser só uma noite, eu acertei
Amar não tava nos meus planos
Não tava procurando, mas eu encontrei
E agora tô aqui falando
Foi o erro mais lindo que eu acertei
Você, você virou o meu mundo de ponta cabeça
Se um dia duvidei do amor, hoje eu tenho certeza
Tiro certo bem na mosca
Eu nem mirei quando acertei sua boca
Tiro certo bem na mosca
Era pra ser só uma noite, eu acertei foi uma vida
Tiro certo bem na mosca
Eu nem mirei quando acertei sua boca
Tiro certo bem na mosca
Era pra ser só uma noite, eu acertei, iê, uma vida toda
Ai ai ai
Era pra ser só uma noite, eu acertei foi uma vida toda
Ai ai ai, ai ai ai
Era pra ser só uma noite, eu acertei
Uma vida toda"""

brazil.lyrics[46] = """Se liga aí, malvada, no que agora eu vou mandar
Um som diferenciado que vai te fazer dançar
JS no comando, vai mandar pra todas elas
Ritmo contagiante, vai entrar na mente delas
Ritmo contagiante, vai entrar na mente delas
Manda pras safadinhas
Evoluiu
Ritmo agressivo, 150 fluiu
Levando levadas que você nunca ouviu
Evoluiu
Vai, brisa na vibe do Kevin o Chris
No tamborzão que te faz mexer
Te faz mexer, te faz mexer
Seja no Lins ou no PPG
Pode começar a des−des−des−des−descer
No tamborzão que te faz mex−mex−mex−mex−mexer
Pode começar a des−des−des−des−descer
No tamborzão que te faz mex−mex−mex−mex−mexer
Eu dou aquela sarradinha
Se liga aí, malvada, no que agora eu vou mandar
Um som diferenciado que vai te fazer dançar
Manda pras safadinhas
Se liga aí, malvada, no que agora eu vou mandar
Um som diferenciado que vai te fazer dançar
JS no comando, vai mandar pra todas elas
Ritmo contagiante, vai entrar na mente delas
Ritmo contagiante, vai entrar na mente delas
Manda pras safadinhas
Evoluiu
Ritmo agressivo, 150 fluiu
Levando levadas que você nunca ouviu
Evoluiu
Vai, brisa na vibe do Kevin o Chris
No tamborzão que te faz mexer
Te faz mexer, te faz mexer
Seja no Lins ou no PPG
Pode começar a des−des−des−des−descer
No tamborzão que te faz mex−mex−mex−mex−mexer
Pode começar a des−des−des−des−descer
No tamborzão que te faz mex−mex−mex−mex−mexer
Eu dou aquela sarradinha
Se liga aí, malvada, no que agora eu vou mandar
Um som diferenciado que vai te fazer dançar
Manda pras safadinhas"""

brazil.lyrics[47] = """Gaiola é o troco
Du Black na voz, na voz, na voz, na voz
Se teu ex não te quis, tem quem queira
Como foi que te perdeu assim, de bobeira?
Que tal dar um tempo de compromisso?
Vem curtir a vida de solteira comigo
Moça, eu não quero me meter na relação
Mas não vale a pena dar moral pra vacilão
Mostra pra ele que hoje tu tá bem demais
E se ele duvidar do que tu é capaz
Então você chuta o balde, nós mete o louco
Se é dia de baile, Gaiola é o troco
Tu vem ficar comigo só por um momento
É proibido se envolver com sentimento
Então vai
Desce, desce, desce, desce, desce com tesão
Senta, senta, senta, vai travando a bucetinha
Sem essa de ficar dividida
Troca o amor por putaria e vem curtir a vida
Então vai
Desce, desce, desce, desce, desce com tesão
Senta, senta, senta, vai travando a bucetinha
Sem essa de ficar dividida
Troca o amor por putaria e vem curtir a vida
Então vai
Se teu ex não te quis, tem quem queira
Como foi que te perdeu assim, de bobeira?
Que tal dar um tempo de compromisso?
Vem curtir a vida de solteira comigo
Moça, eu não quero me meter na relação
Mas não vale a pena dar moral pra vacilão
Mostra pra ele que hoje tu tá bem demais
E se ele duvidar do que tu é capaz
Então você chuta o balde, nós mete o louco
Se é dia de baile, Gaiola é o troco
Tu vem ficar comigo só por um momento
É proibido se envolver com sentimento
Então vai
Desce, desce, desce, desce, desce com tesão
Senta, senta, senta, vai travando a bucetinha
Sem essa de ficar dividida
Troca o amor por putaria e vem curtir a vida
Então vai
Desce, desce, desce, desce, desce com tesão
Senta, senta, senta, vai travando a bucetinha
Sem essa de ficar dividida
Troca o amor por putaria e vem curtir a vida
Então vai
Desce, desce, desce, desce, desce
Senta, senta, senta, vai travando a bucetinha
Sem essa de ficar dividida
Troca o amor por putaria e vem curtir a vida
Gaiola é o troco"""

In [366]:
# Running the clean lyrics function on the text
brazil.lyrics = brazil.lyrics.apply(lambda x: clean_lyrics(x))
brazil.lyrics[0]

'no inicio foi assim terminou tá terminado cada um pro seu lado não precisa ligar mais só que foi eu quem terminou e quem foi largado não espera eu segui minha vida até ela começar seguir a dela  e do meio pro final eu só ia pra onde ela tava cada beijo no rosto que outra boca dava eu morria de raiva e ela tava mais linda cada vez que eu olhava o ciúme não tava batendo tava dando porrada  e eu implorei pra voltar e ela me matou na unha disse que eu tava solteiro eu tava solteiro porra nenhuma implorei pra voltar não me manda embora sou preso na sua vida era só liberdade provisória vai ter que me aceitar de volta  e do meio pro final eu só ia pra onde ela tava cada beijo no rosto que outra boca dava eu morria de raiva e ela tava mais linda cada vez que eu olhava o ciúme não tava batendo tava dando porrada  eu implorei pra voltar e ela me matou na unha disse que eu tava solteiro eu tava solteiro porra nenhuma implorei pra voltar não me manda embora sou preso na sua vida era só liberdade 

In [367]:
# Writing the Brazil lyrics to file
write_to_file(brazil)

'Done!'

In [368]:
# Loading the Germany dataset 
germany_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\Germany Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(germany_path))
germany = pd.read_csv(os.path.basename(germany_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [369]:
# Inspecting the data 
germany = germany[['song_title', 'artist/s', 'country']]
germany['lyrics'] = germany['song_title']
germany.head()

,song_title,artist/s,country,lyrics
0,Du bist mein,"['Zuna', 'Loredana', 'SRNO']",Germany,Du bist mein
1,Allein sein,Ufo361,Germany,Allein sein
2,Blinding Lights,The Weeknd,Germany,Blinding Lights
3,H <3 T E L,"['Dardan', 'Monet192']",Germany,H <3 T E L
4,Puste sie weg,Kontra K,Germany,Puste sie weg


In [370]:
# Getting lyrics for German dataset 
get_lyrics_reboot(germany)

,song_title,artist/s,country,lyrics
0,Du bist mein,"['Zuna', 'Loredana', 'SRNO']",Germany,"SRNO\n\nMarihuana\nWenn ich bei dir bin, geht ..."
1,Allein sein,Ufo361,Germany,"Rich-rich\n\nIch will allein sein, wenn ich me..."
2,Blinding Lights,The Weeknd,Germany,Yeah\n\nI've been tryna call\nI've been on my ...
3,H <3 T E L,"['Dardan', 'Monet192']",Germany,Sie ist bis um drei in mei'm Hotel\nZimmer 442...
4,Puste sie weg,Kontra K,Germany,Liebe in der Luft – puste sie weg\nNeun Millim...
5,Trippin,Luciano,Germany,"Grr\nNegro (Yeah, yeah)\n\nS-Black, fahr' vor,..."
6,Roses - Imanbek Remix,"['SAINt JHN', 'Imanbek']",Germany,Roses\nI walked in the corner with the body sc...
7,Hood,"['AK Ausserkontrolle', 'Shindy']",Germany,"Nur Love für die Hood, free Jusef Axt\nPaff' W..."
8,Breaking Me,"['Topic', 'A7S']",Germany,"La-la-la-la, la-la-la-la\n\nCall me what you w..."
9,Dance Monkey,Tones And I,Germany,"They say, ""Oh my God, I see the way you shine\..."


In [410]:
# Filling in missing values 
germany.lyrics[36] = """Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Shout-out to your mom and dad for makin' you
Standin' ovation, they did a great job raisin' you
When I create, you're my muse
That kind of smile that makes the news
Can't nobody throw shade on your name in these streets
Triple threat, you a boss, you a bae, you a beast
You make it easy to choose
You got a mean touch, I can't refuse (No, I can't refuse it)
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Already pass, you don't need no approval
Good everywhere, don't worry 'bout no refusal
Second to none, you got the upper hand now
Don't need a sponsor, nope, you're the brand now
You're my rock, my Colorado
Get that ring, just like Toronto
Love you now, a little more tomorrow
This how I feel, act like you know that you are
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread (Whip it)
Heart full of equity, you're an asset (Asset)
Make sure that you don't need no mentions (Yeah, yeah)
Yeah, these are my only intentions (Quavo)
No cap, no pretendin', you don't need mentions (No cap)
Got 'em sayin' "goals," they don't wanna be independent ('Pendent)
Tell them to mind your business (Woo), we in our feelings
It's fifty-fifty percentage (Fifty), attention, we need commitment (Oh)
We gotta both admit it (Both), it's funny, we both listen (Both)
It's a blessing (Blessing) 'cause we both get it (Both)
You the best thing (Woo), and I don't need a witness (Best thing)
I'ma find me a ring and pray it's perfect fitted (Perfect, perfect)
Picture perfect, you don't need no filter (No filter)
Gorgeous, make 'em drop dead, you a killer (Oh-oh)
Shower you with all my attention (I will)
Yeah, these are my only intentions (Yeah)
Stay in the kitchen cookin' up, got your own bread (You do)
Heart full of equity, you're an asset (Uh-huh)
Make sure that you don't need no mentions (No mentions)
Yeah, these are my only intentions
Only intentions
That's all I plan to do"""

germany.lyrics[40] = """I love you, baby
And if it's quite all right
I need you, baby
To warm these lonely nights
I love you, baby
Trust in me when I say
I love you 
I love you, baby
And if it's quite all right
I need you, baby
To warm these lonely nights
I love you, baby
Trust in me when I say
I love you
I love you, baby
And if it's quite all right
I need you, baby
To warm these lonely nights
I love you, baby
Trust in me when I say
I love you
I love you, baby
And if it's quite all right
I need you, baby
To warm these lonely nights
I love you, baby
Trust in me when I say"""

germany.lyrics[48] = "Vorbei"

In [493]:
# Applying the clean lyrics function 
germany.lyrics = germany.lyrics.apply(lambda x: clean_lyrics(x))
germany.lyrics[0]

'srno  marihuana wenn ich bei dir bin, geht es dir besser ich bin dein marihuana du rufst mich immer an, wenn du mal stress hast marihuana du hältst an mir fest, denn dir gefällt es marihuana ich bin dein marihuana  marihuana, loredana siehst du auf jedem poster hab mein fahrer, roll mit zwanzig km/h in der großstadt was für labern?! was für karma?! million dollar, steiger umsatz also baby, sag mir, fliegst du mit hoch? halt dich fest, nein, ich lass dich nicht los wir beide sind perfekt, keiner kann uns einholn im sl weinrot drücken wir, im psycho mein prada anzug schneeweiß sie wolln mit mir reden, doch ich sehe keinn nein, ich brauche keinn, der mir den weg zeigt den weg zeigt, mach den weg frei hol das zeug, hau das in das pape rein  marihuana wenn ich bei dir bin, geht es dir besser du bist mein marihuana du rufst mich immer an, wenn du mal stress hast du bist mein marihuana du hältst an mir fest, denn dir gefällt es du bist mein marihuana (du bist mein marihuana, ey)  zuna! mein 

In [426]:
# Loading the Spain dataset 
spain_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\Spain Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(spain_path))
spain = pd.read_csv(os.path.basename(spain_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [427]:
# Inspecting the data
spain = spain[['song_title', 'artist/s', 'country']]
spain['lyrics'] = spain['song_title']
spain.head()

,song_title,artist/s,country,lyrics
0,Safaera,"['Bad Bunny', 'Jowell & Randy', 'Nengo Flow']",Spain,Safaera
1,Tusa,"['KAROL G', 'Nicki Minaj']",Spain,Tusa
2,Amarillo,J Balvin,Spain,Amarillo
3,Rojo,J Balvin,Spain,Rojo
4,La Difícil,Bad Bunny,Spain,La Difícil


In [428]:
# Getting the lyrics 
get_lyrics_reboot(spain)

,song_title,artist/s,country,lyrics
0,Safaera,"['Bad Bunny', 'Jowell & Randy', 'Nengo Flow']",Spain,"Bla, bla, bla, bla, bla, bla\nEy, yo', yo', yo..."
1,Tusa,"['KAROL G', 'Nicki Minaj']",Spain,¿Qué pasa contigo? Dímelo\nRrr!\nO-O-Ovy on th...
2,Amarillo,J Balvin,Spain,"Son más de las doce, nos fuimos de roce\nHoy v..."
3,Rojo,J Balvin,Spain,"(Colores)\nYeah\nJ Balvin, man\nLeggo, leggo\n..."
4,La Difícil,Bad Bunny,Spain,"Ella desaparece, pero aparece cuando le dan ga..."
5,Morado,J Balvin,Spain,"(Colores) Wuh\nYah, leggo\nDespués de tres can..."
6,Tattoo,Rauw Alejandro,Spain,"Tú está' pa' comerte to'a toíta', así está' tú..."
7,Yo Perreo Sola,Bad Bunny,Spain,Ante' tú me pichaba' (Tú me pichaba')\nAhora y...
8,Diosa,Myke Towers,Spain,"(Ey)\n\nEra mi reina, ahora e' mi diosa\nHace ..."
9,Keii,Anuel AA,Spain,Ella ya no piensa en él (En él)\nÉl la convirt...


In [461]:
# Filling in missing values 
spain.lyrics[15] = """Ana Mena
De Palma
Yo le pido al viento
Que te traiga hasta a mí
Solo espero el momento
Para verte pasar
Aunque sea un segundo
Y hacerte saber
Que te quiero invitar a salir
No lo pienses acompáñame
Porque vas a vivir a mi lado
El misterio de un amanecer
Dime si vas a quedarte
Tal vez te pase lo mismo que a mí
Solo se, que yo andaba a oscuras sin ti
Y el camino hacia ti
Se iluminaba
Bajo sonido de una melodía lejana
Los dos bailamos hasta ver la madrugada
Y es que encontrarnos no fue solo por fortuna
Cuando me abrazas siento que mi cuerpo vola, vola
Recógeme te espero a cualquier hora, hora
No quiero pasar esta noche sola
Sin ti mi amor mi mundo es un desierto
La città senza di te
Dime que el amor no tiene ciencia
Dime que el amor no es solo pura coincidencia
Y es que su flecha me atravesó
Rompiendo la coraza de mi corazón
Y quién sabe si estaba escrito
Y acabaremos en el altar
Pasando la noche en una cama matrimonial
Yey eh
Dime si vas a quedarte
Tal vez te pase lo mismo que a mí
Solo se, que yo andaba a oscuras sin ti
Y el camino hacia ti
Se iluminaba
Bajo sonido de una melodía lejana
Los dos bailamos hasta ver la madrugada
Y es que encontrarnos no fue solo por fortuna
Cuando me abrazas siento que mi cuerpo vola, vola
Recógeme te espero a cualquier hora, hora
No quiero pasar esta noche sola
Sin ti mi amor mi mundo es un desierto
La città senza di te
Qué es lo que pasa qué has hecho de mí
Como un embrujo, solo pienso en ti
Lunes o martes, da igual para mí
Siempre es festivo desde que te vi
Vamos a tomarnos a la playa unos tragos
Luego juntitos nos damos un baño
Ponte el bikini más trendy y pa fuera los jeans
Se iluminaba
Como tú te llamas yo no sé pero está bien
Quiero estar contigo en mi cama
Vola, vola
Recógeme te espero a cualquier hora, hora
No quiero pasar esta noche sola
Sin ti mi amor mi mundo es un desierto
La città senza di te"""

spain.lyrics[16] = """Yeh, yeh
Todavía yo te quiero
Pero sé que e' un error
Porque ya tú no me quieres
Y sin ti me va mejor
Y si veo a tu mamá (Ey)
Yo le pregunto por ti
Pa' ver si ya tienes a alguien (¿Uh?)
Alguien que te haga feliz
Y aquí estoy arrebata'o
Pensando en to'a las vece' que te lo metí
Pensando en to'a las vece' que estuve pa' ti (Pa' ti, pa' ti, pa' ti)
No sé por qué diablo' me engaño (No)
Diciendo que te olvidé cuando te extraño
Solo comparto memes, ya yo no escribo nada (Damn)
Y no he borra'o tu foto, solo la puse privada
Maldito año nuevo y lo que me trajo, ey
Me botaron del trabajo por estar mirando pa' abajo
Pensando en ti siempre cabizbajo
Me veo gordito, nada que rebajo (Nah)
No sé por qué la vida me ultrajo (No)
Pensando coger un atajo
Conocí a alguien, pero no sé, nunca encajo (Ah)
Al meno' que sea' tú (Tú, tú, tú, tú)
Baby, te quiero aunque diga lo contrario
Llevo sei' mese' solitario, pero
Hoy salí con lo' muchacho' a beber (Jaja)
Y dije que de ti no iba hablar (No, no, no)
Son las 5, ya va amanecer (Ku-ku-ku-ku)
Si no prenden, la voy a llamar
Hoy salí con lo' muchacho' a beber (Jaja)
Y dije que de ti no iba hablar (No, no, no)
Son las 5, ya va amanecer (Ku-ku-ku-ku)
Si no prenden, la voy a llamar
Todavía yo te quiero (Yo te quiero)
Pero sé que e' un error (Se hace, no)
Porque ya tú no me quieres (No, no)
Y sin ti me va mejor (Juju)
Y si veo a tu mamá (Mami), ey
Yo le pregunto por ti (Okey)
Pa' ver si ya tienes a alguien
Alguien que te haga feliz
Jajaja"""

spain.lyrics[26] = """Le pese a quien le pese
Yo me voy a buscar lo mío
Ahora quieren que me apague
Pero siempre estoy prendío
Si te duele el corazón
Mi hermanito prende un blunt
Yo siempre estoy en modo avión
Caminito pal millón
Estoy buscando dinero, mucho money pa' gastar
Si mi gente está en la calle es porque tienen que josear
Meter dinero en la casa, eso sí es felicidad
Yo no me acuerdo de nadie que no me quería ayudar
Tol día metíos en las escaleras
Buscando billetes, buscando monedas
La calle no es ninguna escuela
Pero si tú quieres aprender, ella te enseña
Que en el barrio se habla por señas
Y que en el bloque no hay contraseñas
La vida es muy perra
Pero yo soy un malo y me río de ella
Sé de donde vengo y eso es lo que me llena
Una familia humilde y un barrio lleno de estrellas
En la calle siempre de to' te entera'
Pero si hablas mucho te saco la lengua fuera
Nosotros no hablamos bulto
Activaos 24 en el punto
Si veo a la po-poli pregunto
Si me busco la vida es porque quiero darme el gusto
Estoy buscando dinero, mucho money pa' gastar
Si mi gente está en la calle es porque tienen que josear
Meter dinero en la casa, eso sí es felicidad
Yo no me acuerdo de nadie que no me quería ayudar
De la zona soy el que más suena
Estaba en la mierda, tol día metío en problemas
Pensaba en gastar en una vida nueva
Pero metío en el barrio porque eso es lo que me llena
Estamos duros como El Chapo
Lo niños crecimos jugamos a ser capos
Trabajamos sin tener contrato
La luz enganchá y maltrato a chivato
Aquí somos lo que ves
No te lo voy a negar
Antes de que pase hambre
Yo me pongo a menear
Mientras mi gente esté bien
No me importa lo demás
Yo solo me la busqué
Cuando no teníamos na
Le pese a quien le pese
Yo me voy a buscar lo mío
Ahora quieren que me apague
Pero siempre estoy prendío
Si te duele el corazón
Mi hermanito prende un blunt
Yo siempre estoy en modo avión
Caminito pal millón
Estoy buscando dinero, mucho money pa' gastar
Si mi gente está en la calle es porque tienen que josear
Meter dinero en la casa, eso sí es felicidad
Yo no me acuerdo de nadie que no me quería ayudar
Pescadería es la zona
No me junto con mariconas
Oye, loco, no me jodas
Quieres ser como yo
Pero la envidia te traiciona
Estoy puesto a josear
No me digas na'
No vine a jugar
Yo vine a ganar
Trá
RVFV
Trá
...en el estudio
... magia"""

spain.lyrics[29] = """White Diamond
Yo te dediqué una de Romeo
Pa' ver si empezamos la aventura
Sé que tú sufres de deseo
Haciéndolo todo eso se cura
Si a la calle yo salgo y te veo con tu tumba'o
No quiero ser exagerado (Exagerado)
Pero me tienes alocá'
Intentando, pero no consigo
Dime, papi, si te viene' conmigo
Decídete, cáele a mi cama, mi cama
Pero me tienes aloca'o
Intentando, pero no consigo (¡duro!)
Dime, baby, si te viene' conmigo (¡qué!)
Dale, nena, cáele a mi cama, mi cama
Desde el primer día tú querías hacérmelo (O-oh)
Y hace tiempo que sé que tú estás intentándolo (O-oh)
Te tengo en espera por si luego me arrepiento (Puh, hmm)
Sé que en el amor tu especialidad es hacer daño (Pra, pra)
Desde el primer día tú querías hacérmelo (O-oh)
Y hace tiempo que sé que tú estás intentándolo (O-oh)
Te tengo en espera por si luego me arrepiento (O-oh)
Sé que en el amor tu especialidad es hacer daño (Po, po, po, po)
A los pretendientes les deja en visto (Piuh, piuh)
A mí me responde y me habla bonito
Si me dice que sí, yo le llego de un brinco (Ah)
Mi tema de La Rubia está pegao' en Tik Tok (Je,je, je)
Y tú ya sabes que te la voy a comer, eh
En tu casa, en la mía, en el hotel (¡Qué!)
Si te pido una pose sé que tú la vas a hacer
Yo sé que ese culo tiene mucho que ofrecer (¡Qué!, ¡qué!, ¡qué!)
Yo te quiero pa' mí, pa' mí (To' pa' mí)
Como la canción de Dalex, baby (¡Qué!)
Esa te la dedico entera pa' ti
Y hacemos el segundo remix tú encima de mí (¡Qué!, ¡qué!, ¡qué!, ¡qué!)
Pero me tienes alocá'
Intentando, pero no consigo (¡Duro!)
Dime, papi, si te viene' conmigo (A-ah)
Decídete, cáele a mi cama, mi cama
Pero me tienes aloca'o
Intentando (¡ah!), pero no consigo (¡Duro!)
Dime, baby, si te viene' conmigo
Dale, nena, cáele a mi cama, mi cama (Piuh, piuh)
Canciones ya te e' dedica'o como siete
Manda fotos tuyas que a mí me dañan la mente
Si tú a mí me llamas te llego como un cohete
Me manda' el diablito y ya me pongo indecente (¡Duro!)
Y yo quiero verte entera desnuda (¡Qué!)
El mejor pretendiente, no te quepa duda (¡Qué!)
Ella quiere sexo y yo tengo la cura (Qué)
Mi bicho en su totito pierde cobertura
Y-y-y-y tú quieres verme entera desnuda
Y sé que quieres chingarme bajo e' la luna
Si tú estás bellaco yo tengo la cura
Tu bicho en mi totito pierde cobertura (A-ah)
Ella está impaciente y no habla de amores
30 mil piedras preciosas tiene el Rolex
Y yo en el estudio grabo, hago que lo enrole
Me gané doscientos mil, coroné, cabrones (Ja-ja)
Pero me tienes alocá'
Intentando, pero no consigo
Dime, papi, si te viene' conmigo (Conmigo)
Decídete, cáele a mi cama, mi cama
Pero me tienes aloca'o
Intentando, pero no consigo (¡Arsa!)
Dime, baby, si te viene' conmigo (Conmigo)
Dale, nena, cáele a mi cama, mi cama (A-ah)
Nadie está mirando (O-oh)
Vamo' a hacerlo, papi, no apagues la luz (Sí)
Me tienes pensando (O-oh)
¿Cómo lo hago pa' que te enamores tú? (Oh)
Nadie está mirando (O-oh)
Vamo' a hacerlo, papi, no apagues la luz (Hmm-hmm)
Me tienes pensando (O-oh)
¿Cómo lo hago pa' que te enamores tú?"""

spain.lyrics[48] = """(Te puse de pantalla en el cel
En mi cabeza tú das vuelta' como carrusel)
Todo pasó así de la nada
Poquito a poco se fue encendiendo una llama
Contigo me envolví, yo ni me lo esperaba
Desde que te probé, mami, yo me enchulé
Todo comenzó como pana'
Contigo encontré to' lo que buscaba
Bebé, tú me hechizaste como un hada
Desde que te probé, mami, yo me enchulé
(Millo Gang)
Te puse de pantalla en el cel
Enchulao' como Alex Rodri está de Jennifer
Te pinto el mundo de colore' sin usar pincel
En mi cabeza tú das vuelta' como carrusel
Nunca te vaya', no haré ninguna falla
Bebé, tú te ganaste la medalla
En mi corazón brincaste la muralla
Voy a luchar por ti, nunca tiraré la toalla
El que te toque se guaya, le prendo la metralla
Aquel te dejó fría como el Himalaya
Yo voy a hacer que te olvide' de ese canalla
Contigo siempre voy a correr por la raya
Mami, tú hace' que sea otro yo
Saca' lo mejor de mí, tú hace' que sea otro yo
Tú me provoca', yo me dejo llevar
Eso no e' normal, pierdo el control cuando me toca'
No lo puedo evitar, me empiezo a erizar, tú besas cabrón
Todo pasó así de la nada
Poquito a poco se fue encendiendo una llama
Contigo me envolví, yo ni me lo esperaba
Desde que te probé, mami, yo me enchulé
Todo comenzó como pana' (Pana')
Contigo encontré to' lo que buscaba
Bebé, tú me hechizaste como un hada
Desde que te probé, mami, yo me enchulé
Yeah, yeah, yeah
No hay otra como tú-ú, tú me da' lu', lu'
Yo estoy pa' ti full, babe, I don't wanna' lose you
Enchula'o, sin mirar pa'l la'o
Nadie me había puesto así, contigo me quedo enjaula'o
Tú eres mi rubí, en la cama ver movie'
Somos inseparable' como Shaggy y Scooby
Como Buzz Light y Woody, pa' mí tú ere' exclusive
Baby, con las otra' nunca, pero tú si (Siempre)
Tú hace' que sea otro yo
Saca' lo mejor de mí, tú hace' que sea otro yo
Tú me provoca', yo me dejo llevar, eso no e' normal
Pierdo el control cuando me toca', no lo puedo evitar
Me empiezo a erizar, tú besas cabrón
(Yeah, yeah, you, yo forever, yeah, yeah, girl, you, yo forever yeah, yeah)
Todo pasó así de la nada
Poquito a poco se fue encendiendo una llama
Contigo me envolví, yo ni me lo esperaba
Desde que te probé, mami, yo me enchulé
Yao'
Dímelo, Antrax"""

In [498]:
# Applying the clean_lyrics function to the lyrics
spain.lyrics = spain.lyrics.apply(lambda x: clean_lyrics(x))
spain.lyrics[0]

'bla, bla, bla, bla, bla, bla ey, yo, yo, yo yo, yo, yo, yo, yah lalalalalalala (blow, blow) lalalalalalala  diablo, qué safaera tú tiene un culo cabrón cualquier cosa que te ponga rompe la carretera (lalalalala; aight) muévelo, muévelo, muévelo, muévelo (lalalalalalala) qué safaera (lalalalala) tiene un culo cabrón cualquier cosa que te ponga rompe la carretera (aight; ¡tra!) muévelo, muévelo, muévelo, muévelo  qué falta de respeto, mami ¿cómo te atreve a venir sin panty? hoy saliste puesta pa mí yo que pensaba que venía a dormir, no vino ready ya, puesta pa una cepillá me chupa la lollipop, solita se arrodilla, hey ¿cómo te atreve, mami, a venir sin panty?  mera, dímelo, dj orma ¿qué tú te cree? jodío cabrón, jeje yo hago lo que me da la gana díselo, conejo ey, ey (jajajaja)  hoy se bebe, hoy se gasta hoy se fuma como un rasta si dio lo permite (si dio lo permite), ey si dio lo permite (que si dio lo permite), ey hoy se bebe, hoy se gasta hoy se fuma como un rasta (woo, woo, woo) si 

In [494]:
# Writing the lyrics to file 
write_to_file(germany)

'Done!'

In [499]:
# Writing the lyrics to file 
write_to_file(spain)

'Done!'

In [500]:
# Loading the Mexico dataset 
mexico_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\Mexico Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(mexico_path))
mexico = pd.read_csv(os.path.basename(mexico_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [501]:
# Inspecting the data 
mexico = mexico[['song_title', 'artist/s', 'country']]
mexico['lyrics'] = mexico['song_title']
mexico.head()

,song_title,artist/s,country,lyrics
0,Safaera,"['Bad Bunny', 'Jowell & Randy', 'Nengo Flow']",Mexico,Safaera
1,Si Veo a Tu Mamá,Bad Bunny,Mexico,Si Veo a Tu Mamá
2,Yo Perreo Sola,Bad Bunny,Mexico,Yo Perreo Sola
3,Rojo,J Balvin,Mexico,Rojo
4,La Difícil,Bad Bunny,Mexico,La Difícil


In [502]:
# Getting the lyrics for Mexico 
get_lyrics_reboot(mexico)

,song_title,artist/s,country,lyrics
0,Safaera,"['Bad Bunny', 'Jowell & Randy', 'Nengo Flow']",Mexico,"Bla, bla, bla, bla, bla, bla\nEy, yo', yo', yo..."
1,Si Veo a Tu Mamá,Bad Bunny,Mexico,NaN
2,Yo Perreo Sola,Bad Bunny,Mexico,Ante' tú me pichaba' (Tú me pichaba')\nAhora y...
3,Rojo,J Balvin,Mexico,"(Colores)\nYeah\nJ Balvin, man\nLeggo, leggo\n..."
4,La Difícil,Bad Bunny,Mexico,"Ella desaparece, pero aparece cuando le dan ga..."
5,Tusa,"['KAROL G', 'Nicki Minaj']",Mexico,¿Qué pasa contigo? Dímelo\nRrr!\nO-O-Ovy on th...
6,Blinding Lights,The Weeknd,Mexico,Yeah\n\nI've been tryna call\nI've been on my ...
7,Ignorantes,"['Bad Bunny', 'Sech']",Mexico,"Baby, este es Sech\nBad Bunny, baby (Yeah)\nQu..."
8,Morado,J Balvin,Mexico,"(Colores) Wuh\nYah, leggo\nDespués de tres can..."
9,Sigues Con El,"['Dimelo Flow', 'Arcangel', 'Sech']",Mexico,Si tú te vuelves loca por mí (Por mí)\nY yo me...


In [503]:
# Filling in missing values 
mexico.lyrics[1] = """Yeh, yeh
Todavía yo te quiero
Pero sé que e' un error
Porque ya tú no me quieres
Y sin ti me va mejor
Y si veo a tu mamá (Ey)
Yo le pregunto por ti
Pa' ver si ya tienes a alguien (¿Uh?)
Alguien que te haga feliz
Y aquí estoy arrebata'o
Pensando en to'a las vece' que te lo metí
Pensando en to'a las vece' que estuve pa' ti (Pa' ti, pa' ti, pa' ti)
No sé por qué diablo' me engaño (No)
Diciendo que te olvidé cuando te extraño
Solo comparto memes, ya yo no escribo nada (Damn)
Y no he borra'o tu foto, solo la puse privada
Maldito año nuevo y lo que me trajo, ey
Me botaron del trabajo por estar mirando pa' abajo
Pensando en ti siempre cabizbajo
Me veo gordito, nada que rebajo (Nah)
No sé por qué la vida me ultrajo (No)
Pensando coger un atajo
Conocí a alguien, pero no sé, nunca encajo (Ah)
Al meno' que sea' tú (Tú, tú, tú, tú)
Baby, te quiero aunque diga lo contrario
Llevo sei' mese' solitario, pero
Hoy salí con lo' muchacho' a beber (Jaja)
Y dije que de ti no iba hablar (No, no, no)
Son las 5, ya va amanecer (Ku-ku-ku-ku)
Si no prenden, la voy a llamar
Hoy salí con lo' muchacho' a beber (Jaja)
Y dije que de ti no iba hablar (No, no, no)
Son las 5, ya va amanecer (Ku-ku-ku-ku)
Si no prenden, la voy a llamar
Todavía yo te quiero (Yo te quiero)
Pero sé que e' un error (Se hace, no)
Porque ya tú no me quieres (No, no)
Y sin ti me va mejor (Juju)
Y si veo a tu mamá (Mami), ey
Yo le pregunto por ti (Okey)
Pa' ver si ya tienes a alguien
Alguien que te haga feliz
Jajaja"""

mexico.lyrics[32] = """Qué se siente ser feliz
Solamente cuando tomas
No llenar ese vacío
Por que únicamente es mío
Por eso es que te abandonan
Y es que siempre ha sido así
Lo que quiero lo consigo
A tu nombre alzo mi copa
Porque dormiré con otra
Y tú soñaras conmigo
Te lo dije corazón
Quien lastima está cabrón
Siempre tiene su castigo
Dime cómo quieres que te explique
Que por más que llores y supliques
Yo ya no vuelvo contigo
La verdad sí me arrepiento
Por haberte conocido
Y vete porque causas pena ajena
Creo que en realidad tienes problemas
Cuánto quieres que te pague
Solo quiero que te largues
Te lo doy en efectivo
Échele mi compa Lenin Ramírez
Y así suena Grupo Firme, viejo
¡Salud!
Dime cómo quieres que te explique
Que por más que llores y supliques
Yo ya no vuelvo contigo
La verdad sí me arrepiento
Por haberte conocido
Vete porque causas pena ajena
Creo que en realidad tienes problemas
Cuánto quieres que te pague
Solo quiero que te largues
Te lo doy en efectivo"""

In [504]:
# Applying clean_lyrics function to Mexico 
mexico.lyrics = mexico.lyrics.apply(lambda x: clean_lyrics(x))

In [556]:
# Writing the lyrics to file 
write_to_file(mexico)

'Done!'

In [557]:
# Loading the Italy dataset 
italy_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\Italy Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(italy_path))
italy = pd.read_csv(os.path.basename(italy_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [558]:
# Inspecting the data 
italy = italy[['song_title', 'artist/s', 'country']]
italy['lyrics'] = italy['song_title']
italy.head()

,song_title,artist/s,country,lyrics
0,Auto Blu,"['Shiva', 'Eiffel 65']",Italy,Auto Blu
1,"SPORT + muscoli (RMX) (feat. Lazza, Paky, Luch...","['Marracash', 'Luche', 'Lazza', 'Paky', 'Taxi B']",Italy,"SPORT + muscoli (RMX) (feat. Lazza, Paky, Luch..."
2,Bando,ANNA,Italy,Bando
3,Fiori di Chernobyl,Mr.Rain,Italy,Fiori di Chernobyl
4,Good Times,Ghali,Italy,Good Times


In [559]:
# Getting the lyrics for Italy 
get_lyrics_reboot(italy)

,song_title,artist/s,country,lyrics
0,Auto Blu,"['Shiva', 'Eiffel 65']",Italy,"Ehi, ehi, ehi, ehi, ehi, ehi, eh-ehi\nLa mia s..."
1,"SPORT + muscoli (RMX) (feat. Lazza, Paky, Luch...","['Marracash', 'Luche', 'Lazza', 'Paky', 'Taxi B']",Italy,"Sport\n\nAh, faccio soldi per sport\nCol telef..."
2,Bando,ANNA,Italy,"Ehi, Anna\n\nCi beccavamo nel bando, sopra il ..."
3,Fiori di Chernobyl,Mr.Rain,Italy,La libertà spaventa più di una prigione\nE tut...
4,Good Times,Ghali,Italy,"Sembra la fine del mondo, ma mi calma\nMi chie..."
5,Blinding Lights,The Weeknd,Italy,Yeah\n\nI've been tryna call\nI've been on my ...
6,Per sentirmi vivo,"['Fasma', 'GG']",Italy,NaN
7,Boogieman (feat. Salmo),"['Ghali', 'Salmo']",Italy,"A Baggio quando piove, piove un botto (Un bott..."
8,blun7 a swishland,tha Supreme,Italy,"Yeah\n\nSwisho un blunt a Swishland, bling bla..."
9,Snitch E Impicci (con FSK SATELLITE),"['DrefGold', 'FSK SATELLITE']",Italy,NaN


In [561]:
# Filling in missing values 
italy.lyrics[6] = """Non ti voglio più scrivere
Dirti come vivere
Per non farti piangere ho fatto l’impossibile
Pensi non sia fragile
Pensi sia incredibile
Perchè non scendon lacrime quando scendono a te
Ma a te sembra facile
Dirti che sto bene quando tutto non va
Ed è brutto stare insieme
Perchè scordo che era e il ricordo mi fa male
Del rapporto che c'era prima di questa canzone io e te
Cosa siamo diventati io e te
Sono quello che odiavi di me
Baby perchè non mi ami amore sbatti le ali e vola via da me
Via da me via da te via da questa città
Via da noi via da te e domani chissà
Questa fama questa luce questa notorietà
Non mi basterà non mi servirà
E se dentro muoio lento sai che fuori sorrido
Io ti ho persa dentro il letto per tenerti vicino
Io ti ho visto per la strada e sai che ti voglio in giro
E se adesso non mi parli è perchè in testa ho un casino
E' per questo ti scrivo
Oggi non mi parli perchè infondo ti uccido
Perchè infondo mi uccidi
Perchè infondo mi uccidi
Siamo uguali opposti con i cuori divisi
Oggi dove sei non lo so
Ieri eri tutto ciò che avevo io
Oggi chi sei non lo so
Abbiamo detto basta senza dirci addio
Ma a te sembra facile
Dirti che sto bene quando tutto non va
Ed è brutto stare insieme
Perchè scordo che era e il ricordo mi fa male
Del rapporto che c'era prima di questa canzone io e te
Cosa siamo diventati io e te
Sono quello che odiavi di me
Baby perchè non mi ami amore sbatti le ali e vola via da me
Via da me via da te via da questa città
Via da noi via da te e domani chissà
Questa fama questa luce questa notorietà
Non mi basterà non mi servirà
E se dentro muoio lento sai che fuori sorrido
Io ti ho persa dentro il letto per tenerti vicino
Io ti ho visto per la strada e sai che ti voglio in giro
E se adesso non mi parli è perchè in testa ho un casino
Io ti ho nella testa
Tu mi hai detto resta
E vola via ma fallo con me
Io ti ho detto basta
Dai ti prego scappa
Ma fallo lontano da me"""

italy.lyrics[9] = """Chiello
Non voglio snitch nel gang
Quando l’ho fatto pensavo al guadagno
Non voglio snitch nel gang
Quando parlavo chi stavi ascoltando?
Non voglio snitch nel gang
Stiamo pensando al cash
Tutti gli sbagli li sto pagando
Tornare indietro, non posso farlo
Chiello
Non voglio snitch nel gang
Quando l’ho fatto pensavo al guadagno
Non voglio snitch nel gang
Quando parlavo chi stavi ascoltando?
Non voglio snitch nel gang
Stiamo pensando al cash
Tutti gli sbagli li sto pagando
Tornare indietro, non posso farlo
DrefGold
Eh-eh-eh, non voglio snitch nel gang
Quello è un bitch ass, so che parla di me (Uh)
Eh-eh-eh (Yah), lean e no Moët
Fanta arancione è diventata red
Voglio avere soldi in tasca
Lei fuma sopra i miei rasta (Eh)
Big racks dentro la banca
Brucia lento il mio Back’, non c’è (No)
Nella mia gang chi parla (Eh)
Ho Cali e tre Hi-Tech in palla (In palla)
Il tuo chain è ferraglia
Non sei FSK, non sei una Kanaglia, gang
Chiello, Sapobully & Taxi B
Non voglio snitch nel gang
Quando l’ho fatto pensavo al guadagno
Non voglio snitch nel gang
Quando parlavo chi stavi ascoltando? (Baing, baing, baing, baing)
Non voglio snitch nel gang
Stiamo pensando al cash
Tutti gli sbagli li sto pagando
Tornare indietro, non posso farlo (Gang, gang, gang, gang)
Taxi B
Ogni mese un nuovo Moncler (Moncler), mi
Alzo di piano, alto (Alto)
Come il Monte Bianco (Bianco)
No più fresca in tasca (Bang, bang)
Ogni mese un nuovo Moncler, mi
Alzo di grado, un anno, quasi maresciallo (Gang, gang)
Non dirlo se poi non puoi farlo
Alla fine l’abbiam rapinato (Bang, bang, bang, bang)
Ah, ogni giorno è come se non vivo prima (Gang, gang)
A malapena cammino
Ho preso quella roba viola
FSK + Gigi Komparema
Siamo la trap in Italia
Sapobully & DrefGold
Komparemo Gigi (Gigi)
Facevamo impicci (Ollare gang)
Avevamo già Gucci (Gucci)
Ma facendo flexing
Brothmo Bully (Ah)
Giocavamo a Bully (4L)
Adesso siamo belli (Ah, skrrt)
Adesso siamo furbi (Brr-pow)
FSKNGL (Gang)
Guarda, mamma, faccio cose belle (Ollare gang)
Sto facendo soldi veramente (Esk’)
Questi bla-bla non sanno niente (Let’s go)
Questi bla-bla non sanno le cose
Questi bla-bla vendono le rose (Baing-baing)
Sono rose pure le mie droghe (Brr-pow)
Guarda, mamma, non c’ho più la tosse
Bitch lo sa che sono 4L (4L)
Faccio un chain, FSKNGL (Big chain)
Eravamo plug, adesso siamo star (Star, okay)
Lo sanno tutti, stiamo andando up
Sottovuoto nella trappola
Eminflex il mio materasso
Io sono magro, no grasso
Gas come la macchina (Skrrt)
Pam-pam, ra-ta-ta-ta
Fa-fa: “Tu-tu”, ah-ah
Lei studia matematica
Perché nella gang si fa algebra
Infatti adesso ho grandi tasche
Letra lyrics lyric letras versuri musiek lirieke tekstet paroles"""

italy.lyrics[16] = """Taxi B
Greg Willen non dormire
Ansia no, ho fatto due soldi, ho detto: “Bianca stop”
Dopo due secondi Polo Nord
A casa c’è il Polo Nord e appresso c’ho il Polo Nord
Dentro il bomber di Moncler che fa più luce ho il Polo Nord
Taxi B
E sai che ho detto: “Ansia no”, ho fatto due soldi, ho detto: “Bianca stop”
Dopo due secondi Polo Nord
A casa c’è il Polo Nord e appresso c’ho il Polo Nord
E dentro il bomber di Moncler che fa più luce ho il Polo Nord
E sai che ho detto: “Ansia no”, ho fatto due soldi, ho detto: “Bianca stop”
Dopo due secondi il Polo Nord
A casa c’è il Polo Nord e appresso c’ho il Polo Nord
E dentro il bomber di Moncler che fa più luce ho il Polo Nord
Sapobully & Taxi B
Quando entro nel club sembra il Polo Nord (gang)
Polo Ralph Lauren (Ralph Lauren), Amiri o Balmain (euro)
Sto fumando cookie da 5 euro al g
Stelle Givenchy (baing)
Ciabattine Armani (Armani), Taxi B, Gucci e Fendi (4L)
Spostavamo pacchi (gang), ora alziamo stipendi (gang)
Siamo i boss del trapshit (ollare gang)
Ollare the gang (pa-pa-pa-pa, gang)
Taxi B
Parlo Carlo Cracco (ehi), sip-sippo lean, ehi
Per la tua tipa sono benzodiazepina, ehi (uoh)
La bagno come un nachos, mi, mi, mi
Mi chiama “gotico”, se poi è un pompino, ho sprecato un emoticon
Oppure due o tre (no), oppure che vuoi? (ehi)
Sai che lo faccio col gang (gang), mica lo faccio con te (te)
Ta-ta-taxi e FSK uguale la trap in Italia (Italia)
Ne abbiamo chiusi più 50 in banca solamente al primo album
Taxi B
A casa c’è il Polo Nord e appresso c’ho il Polo Nord
E dentro il bomber di Moncler che fa più luce ho il Polo Nord
Taxi B
E sai che ho detto: “Ansia no”, ho fatto due soldi, ho detto: “Bianca stop”
Dopo due secondi il Polo Nord
A casa c’è il Polo Nord e appresso c’ho il Polo Nord
E dentro il bomber di Moncler che fa più luce ho il Polo Nord
Chiello
Fuck Sprite clean, non raccolgo il cash (cash)
Drip dentro al club (ehi, ehi, ehi, ehi, ehi), con la gang, siamo i boss (drip)
1, 2, 3 (ehi), bianca stop (ehi), ansia no (no)
Mi inseguono bitch, ice, nuovo Moncler (tu-tu-tu-tu-tu-tu-tu-tu)
Non ho più freddo, mamma (gang, gang), no più ansia, grazie (gang, gang, gang, gang)
Madonna, redimi i peccati, grazie (bang, bang)
F-S-K (tu-tu), sto lanciando il cash (pew, pew, pew, pew)
Back nel privé, double cup, siamo i boss
Taxi B
A casa c’è il Polo Nord e appresso c’ho il Polo Nord
E dentro il bomber di Moncler che fa più luce ho il Polo Nord
Letra lyrics lyric letras versuri musiek lirieke tekstet paroles"""

italy.lyrics[26] = """Cento, ne hai fatti cento
Ma sei un bel fesso
Di quei cento ti levo cento, ah
Certo, rimani a secco
Non mi interesso
Sento, ma non mi sento al 100%
Riesco, cappuccio 'n testa, me sento fresco
Quindi esco fresco de' testa, come al campetto
Vento, giri col vento, io 'n ce riesco
Venti, giramo 'n venti sotto a 'n ombrello
Senti, mica me senti? Mica ce penso
Ma ce pensi? Mica ce pensi a quello che penso
Pezzi, dice c'hai i pezzi, ma 'n c'ha 'r cervello
Persi, ragazzi persi dentro ar campetto
E siamo cresciuti a fa' le stesse cose
Abbiamo rubato giusto qualche cosa
I vicoli pe' strada ricordano il nome
Le cene pagate grazie a qualche sola
E siamo cresciuti a fa' le stesse cose
Abbiamo rubato giusto qualche cosa
I vicoli pe' strada ricordano il nome
Le cene pagate grazie a qualche sola
C'è qualche problema amico, Bugs Bunny
Bastardi, vorrei bastarti
Ma sai, da grandi
Dar grammi fa danni
T'accasci ed accanni
Si è stanchi se campi da cani
Taccagni i cantanti
Campati sui prati nei viali innevati
Ne evadi se vali
Balli, combatti coi patti più chiari
Abbatti Bambi, da bambino piangi
Ma la vita è sbatti
Sporca e poi non lava i piatti
Genova-Roma, compatti
Sai che ormai siamo accoppiati
Quali flow copiati
Vai in cella cent'anni
Bella pe' Gianni
E siamo cresciuti a fa' le stesse cose
Abbiamo rubato giusto qualche cosa
I vicoli pe' strada ricordano il nome
Le cene pagate grazie a qualche sola
E siamo cresciuti a fa' le stesse cose
Abbiamo rubato giusto qualche cosa
I vicoli pe' strada ricordano il nome"""

italy.lyrics[30] = """Più stanno a insultare sotto più sto a fare bingo
Anche questo mese sto facendo live in giro
Non aver paura del mio giro, che è un bel giro (Cash)
Solo che sgamiamo se sei vero o se fai il mimo (Uoh)
Anche questo disco, sì, ci sono uscito vivo (Uoh)
Suona troppo fresco, no competitor, è il minimo (Uoh)
Sto con la Machete e sono benedetto a loro (Ah)
Tu sei un pollo, fra', staresti bene con l'alloro (Grr)
Uh, ehi, hai fatto cose (Ah)
Uh, non me le dire (No), uh, facciamo un gioco
Mhm, yeah, ehi, dimmi chi rimane (No), ehi (Chi?)
Siamo la family, quella invincibile (Uh, wow), yeah
Facciamo hit e rompiamo classifiche (Uh, wow), yeah
Se ci hai dissato, fra', scelta terribile (Uh, wow), yeah
Ad ogni pezzo, fra', un missile, missile (Wow), yeah
Fra', voglio che salti
Fammelo vedere, però non che parli
Poi non lo fai neanche con i sogni in grande (Yah, yah)
Giro tutti i palchi, però stai a insultarmi (No)
Niente va mai bene (No), il tipo non mi piace
Puoi fare meglio (Yeah, yeah, yeah, yeah)
Non mi stai convincendo (Yeah, yeah, yeah, yeah)
Per me ti stai vendendo (Yeah, yeah, yeah, yeah)
Sei cambiato in peggio (Yeah, yeah, yeah, yeah)
Ho fatto il giro del mondo (Vraun)
E mi è sembrato un secondo (Yeah)
Con me c'è mancato poco (Rrah)
Il tuo flow non cera e c'è mancato poco (Wow)
Se mi chiami, vengo a nuoto (Wow)
Il pianeta esplode, però poi ci penso dopo (Wow)
Oggi sono in ansia perché esce la box logo (Wow)
Scemo io che penso a prender casa col box nuovo (Wow)
Qui tante formiche, però mai visto una Zeta
L’insetto più è piccolo e più pensa alla moneta (Wow)
Infatti chi stampa i soldi è chiamata zecca (Wow)
Bravo, fai biliardo (Bravo), fumati una stecca (Yeah)
A scuola entravo in ritardo, ma per gentilezza (Yah)
Avevo in testa solo caldo quando stavo al freddo (Yah)
Se mi guardo dentro (Yah)
Ho un campo di opportunità per cogliere l'attimo
Ma devo farlo adesso, quindi (Uoh, uoh, uoh, uoh)
Yeah, fra', voglio che salti
Fammelo vedere, però non che parli
Poi non lo fai neanche con i sogni in grande
Giro tutti i palchi, però stai a insultarmi (No, wow)
Niente va mai bene (No), il tipo non mi piace
Puoi fare meglio (Yeah, yeah, yeah, yeah)
Non mi stai convincendo (Yeah, yeah, yeah, yeah)
Per me ti stai vendendo (Yeah, yeah, yeah, yeah)
Sei cambiato in peggio (Yeah, yeah, yeah, yeah)"""

italy.lyrics[31] = """Sick Luke, Sick Luke
Ah-ah, la strada è nostra
Ah-ah, l'ultima volta
Ah-ah, la strada è nostra
Ah-ah, l'ultima volta
Lo sai che io non parlo?
Su me c'ho puntato
So' stato pe' strada
E lo sai che io parlo la lingua del branco
E fateme santo, me sento a disagio
Co' tuo padre accanto
Mentre che mi parla dei viaggi che ha fatto
Io stato solo in piazza
E grazie alla musica sono rinato
Voglio vola' in alto
È grazie alla musica se ora mi pagano
E giro l'Italia
Per la prima volta ho visto un contratto
Mi sono tuffato
Pensavi di essere un grande contatto
Ma chi te contatta?
Ah-ah, la strada è nostra
Ah-ah, l'ultima volta
Sapivi ca 'nzieme a 'na femmina
Si 'a tratt' bono primma o poi te lascia
Staje male, tu ancora 'a vvuò bene
P' te er 'tt cos' ma 'tt cos' passa
'Nu napulitano corrett perdono
Nz scorda re sbagl' che e fatto
T'arricuordi erm una persona
Ce sta 'nu legame 'nto sang' e nun batt'
Nun m'incolpà si t' facc' nu sgarro
Semb' 'a mazzetta si trase 'ndo bar
Papà me ricette c'ha facc' a fa' 'a famm
'Nu mes' ropp so' asciuto cantante
Primm vrev l'orario 'ngopp 'o schermo rutto
Pò primma 'e l'iPhone o Huawei
Via Calabritto 'o saje parlo dialetto
Rind' addu Monetti m'accatto 'nu Date (Brr)
Ah-ah, la strada è nostra
Ah-ah, l'ultima volta
Ah-ah, la strada è nostra
Ah-ah, l'ultima volta"""

italy.lyrics[34] = """E Poison ngopp o beat sfonn e cass fra’
Tiengo un amigo, che me habla Narcos
Da Secondigliano, a Santo Domingo (ngo, ngo)
Abbiamo fatto goal (goal, goal), abbiamo fatto gol
In giro per Milano, vesto Valentino
Lei mi guarda da lontano e dice che soy fuego, fuego, fuego, fuego, fuego, fuego, fuego, fuego, fuego
A 180 a Miami, Danny portami un’Audi
Vogliono farmi fuori anche gli Americani
Ma io glielo detto, sono Italiano
Forse non mi avrà capito quindi gli ho sparato
Non hablo contigo se non sei mio amigo (traduzione: non parlo con te se non sei mio amico)
Non hablo contigo se non sei mio amigo
Non hablo contigo se non sei mio amigo
Teng e sord assai, comm facc n’to dic (traduzione: ho tanti soldi, come faccio a non dirtelo)
Nun mov nu chil’ o saij mov’ nu pacc’
Si nu pupazz’, gir’ ch’ pazz’
Scriv’ nu piezz’ rind a primma class’ e ropp fa nu milion e fatturat’
Guard che fann
Nun teng na gang, nun teng na fam ma teng nu clan
Semp che tut o sai pur e rummenc però apparat o milion nta banc
O sacc’ sa fiss’, ven na ser e po vo semp o bis
Chius’ ‘nta suite facc business
Nziem a na tip ca addor e Van Cleef
Rind o mac teng’ l’invid’ e ‘tt e rapper
Stong che ninja rind a nu Range
Teng’ a n’amic che sta ‘nta cell
Aspe’ sta facenn a guerr’
Agg appis quatt fest pecchè teng nu summit
No chillu là nziem’ e guapp ma co Papa e che ministr’
Agg appicciat’ nu blunt ‘nta stanz’ cu nu sceicc’
Agg fatt’ bust e sord sul cu’ ‘na penna Bic
Non sposto un chilo, sposto pacchi
Sei un pupazzo, giri con i pazzi
Scrivo un pezzo in prima classe e ci faccio un milione di fatturato
Guarda che fanno
Non hanno una gang, non hanno fama ma un clan?
So che guarda, viene una sera e poi vuole sempre il bis
Chiuso nella suite faccio affari
Insieme a una tipa che adora i Van Cleef
Nel mac ho l’invidia di tutti i rapper
Sto con i ninja dentro un Range
Ho un amico che sta in prigione
Aspetta sta facendo la guerra
Ho fatto qualche festa perché ho un summit
Non insieme ai coraggiosi ma col Papa e coi Ministri
Ho acceso un sigaro nella stanza con uno sceicco [Nota: il blunt è un sigaro fatto di cannabis]
Ho fatto tanti soldi solo con una penna bic
Non hablo contigo se non sei mio amigo
Non hablo contigo se non sei mio amigo
Non hablo contigo se non sei mio amigo
Teng e sord assai, comm facc n’to dic
Nu chil’ r’or nguoll fra m sent comm a Ramses
Cu doij cuban’ nguoll e nun ce parl’ facc’ sol’ sex (sex)
E guaglion già pront a fa fuego
Si te mann’ nto dic’ te truov’ nda cas’ che passamontagna, hasta luego
Nun m vir sto nciel’ che nuvl e stell in estate, mai grazie a nisciun
Cu nu par’ e cumpagn’ la for già pront ca 9 a 28 a 31
Tutt sti sciem nun avn’ idea (Tutt sti sciem nun avn’ idea)
Parln tropp’ nun fann p’ me
Vonn fa e guapp ma fann’ a vre
A 180 orarij ngopp a l’Asse Median
Teng o cuntatt’ buon e so ‘tt American
Tenimm l’uocchie nguoll e’ tutt’ a scen’ Italian’
Vuò sape comm facc e sord, to dic riman
Un chilo d’oro addosso, mi sento come Ramses
Con due cubane addosso con cui non parlo, ci faccio solo sesso
I ragazzi già pronti a sparare
Se te li mando non te lo dico, te li ritrovi in casa con passamontagna, addio
Non mi vedi, sto in cielo con le nuvole e le stelle in estate, mai grazie a nessuno
Con un paio di compari la fuori già pronti con la 9 a 28 a 31
Tutti questi scemi non ne hanno idea
Parlano troppo, non fanno per me
Vogliono fare i coraggiosi ma fanno finta di esserlo
A 180 km all’ora sull’Asse Mediano
Ho ottimi contatti e sono tutti americani
Abbiamo gli occhi addosso è tutta scena italiana
Vuoi sapere come faccio i soldi? Te lo dico domani
Non hablo contigo se non sei mio amigo
Non hablo contigo se non sei mio amigo
Non hablo contigo se non sei mio amigo
Teng e sord assai, comm facc n’to dic"""

italy.lyrics[39] = """Io questo week-end
Saluto tutti e me ne vado vado a Courmayeur
Salgo al rifugio e chiamo un altro sommelier
Tre mila metri, sboccio il cristal sul dessert
Portatemi da bere
La la la la la
Balliamo sulla neve
La la la la la
Riempitemi il bicchiere
La la la la la
Balliamo sulla neve
La la la la la
Come tutte le sere
Stanotte vado a Courma
Vado a Courma
Ci salgo ubriaco con il quad
Tu con il pullman
Ballo a dieci sotto zero
Con il bicchiere sempre pieno
Se non ritorno a casa sclero
Mamma ho perso l'aereo
Whoo-oah
Quant'è fredda sta notte
Whoo-oah
Ci scaldiamo con i cocktail
Whoo-oah
Voglio perdere la voce cantando whoo-oah
Whoo-oah
Io questo week-end
Saluto tutti e me ne vado vado a Courmayeur
Salgo al rifugio e chiamo un altro sommelier
Tre mila metri, sboccio il cristal sul dessert
Portatemi da bere
La la la la la
Balliamo sulla neve
La la la la la
Riempitemi il bicchiere
La la la la la
Balliamo sulla neve
La la la la la
Come tutte le sere
Mischio tutto, rum e volta
Sono ubriaco, un'altra vodka
Voglio ancora un bombardino
Accendo l'iqos con l'accendino
C'è la mia tipa che sbraita
Che sbraita
E dai riportami in baita
In baita
Whoo-oah
Quant'è fredda sta notte
Whoo-oah
Ci scaldiamo con i cocktail
Whoo-oah
Voglio perdere la voce cantando whoo-oah
Whoo-oah
Io questo week-end
Saluto tutti e me ne vado vado a Courmayeur
Salgo al rifugio e chiamo un altro sommelier
Tre mila metri, sboccio il cristal sul dessert
Portatemi da bere
La la la la la
Balliamo sulla neve
La la la la la
Riempitemi il bicchiere
La la la la la
Balliamo sulla neve
La la la la la
Come tutte le sere"""

italy.lyrics[47] = """Oh Andry
Yeah, se fumo è la prassi
Non volevo andarci a scuola
Sapevo già da allora, ho dato la parola
Parlano i fatti e in una lingua sola
Ho fatto bingo e brotha, tu hai fatto la tua storia
Tu credi mi basti, uscire fuori a cena
E non preoccuparmi, se ho banconote o meno
Se si toglie i tacchi, ho già capito il nesso
Fra’ tu arrivi tardi, devi cambiare mezzo
E ti rispondo male di getto, tu non parli nemmeno
Forse è colpa dell'ego, ma te l'avevo detto
Non ho manco più tempo, che tu ci creda o no
Ma sai che se prometto, ti do
Babe ti do la mia parola
Che ce ne andremo via di qua
Come quando ti aspettavo sotto l'acqua fuori scuola
Con un milione d'idee, senza biglietto del tram
Io ti do la mia parola
Anche se non so più parlare, yeah
Babe ti do la mia parola, ma tu devi saperla usare, yeah
Ormai non mi fa male manco l'amore, ehi
Forse mi sento immortale, Marco D'Amore
Sembri pericolosa con quel ferro sul culo
Anche se è un tattoo, sembra vero, lo giuro
Sai perché sto nel culo a tutti questi del rap
Supero tutti da destra perché no, non si fa
Stavo online finchè non avevo giga nel cell
Ora sembro la Gioconda, fai la fila per me
Ho del cream nelle tasche del miei nuovi Amiri
Con l'elastico o li lascio in giro tra i miei deliri
Forse i soldi fanno stare una persona da sola
Se ti incontrerò per caso ti dirò
Babe ti do la mia parola
Che ce ne andremo via di qua
Come quando ti aspettavo sotto l'acqua fuori scuola
Con un milione d'idee, senza biglietto del tram
Io ti do la mia parola
Anche se non so più parlare, yeah
Babe ti do la mia parola, ma tu devi saperla usare, yeah
Oh no, baby, non dirmi che mi cercavi
Mi ricordo quando affondavo con gli altri schiavi
Avevo un sogno, gli altri li credevi più bravi
Ora guardo il conto, giuro, non sai quanto sbagliavi
Ovunque mi muovo (ehi), tipe arrivano a casa come un Glovo (Brr)
Poi li spendiamo assieme in ogni modo (Cash, cash, cash)
Perché da solo, no, non me li godo (No, no)
Ho un nodo stretto in gola
Che sia il senso di colpa oppure il fumo, non lo so
Meglio qui che sul tuo letto e sola
Non ti sarò fedele, forse no, però
Babe ti do la mia parola
Che ce ne andremo via di qua
Come quando ti aspettavo sotto l'acqua fuori scuola
Con un milione d'idee, senza biglietto del tram
Io ti do la mia parola
Anche se non so più parlare, yeah
Babe ti do la mia parola, ma tu devi saperla usare, yeah"""

In [562]:
# Applying the clean lyrics function 
italy.lyrics = italy.lyrics.apply(lambda x: clean_lyrics(x))

In [613]:
# Writing the lyrics to file
write_to_file(italy)

'Done!'

In [9]:
# Importing the Philippines data 
philippines_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\Philippines Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(philippines_path))
philippines = pd.read_csv(os.path.basename(philippines_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [10]:
# Inspecting the data 
philippines = philippines[['song_title', 'artist/s', 'country']]
philippines['lyrics'] = philippines['song_title']
philippines.head()

,song_title,artist/s,country,lyrics
0,Intentions (feat. Quavo),"['Justin Bieber', 'Quavo']",Philippines,Intentions (feat. Quavo)
1,Don't Start Now,Dua Lipa,Philippines,Don't Start Now
2,Imahe,Magnus Haven,Philippines,Imahe
3,Make It With You,Ben&Ben,Philippines,Make It With You
4,Sunday Best,Surfaces,Philippines,Sunday Best


In [11]:
# Getting lyrics for the Philippines 
get_lyrics_reboot(philippines)

,song_title,artist/s,country,lyrics
0,Intentions (feat. Quavo),"['Justin Bieber', 'Quavo']",Philippines,"Imagen perfecta, no necesitas filtro\nMagnífic..."
1,Don't Start Now,Dua Lipa,Philippines,"If you don't wanna see me\n\nDid a full 180, c..."
2,Imahe,Magnus Haven,Philippines,Kinukulayan ang isipan\nPabalik sa nakaraan\n'...
3,Make It With You,Ben&Ben,Philippines,"Hey, have you ever tried\nReally reaching out ..."
4,Sunday Best,Surfaces,Philippines,"Good, feeling good\n\nAyy, feeling good, like ..."
5,death bed (coffee for your head) (feat. beabad...,"['Powfu', 'beabadoobee']",Philippines,"Don't stay awake for too long, don't go to bed..."
6,Pagtingin,Ben&Ben,Philippines,Dami pang gustong sabihin\nNgunit 'wag nalang ...
7,Dance Monkey,Tones And I,Philippines,"They say, ""Oh my God, I see the way you shine\..."
8,Sa Susunod na Habang Buhay,Ben&Ben,Philippines,Kaya namang makayanan\nKahit pa na nahihirapan...
9,Someone You Loved,Lewis Capaldi,Philippines,"I'm going under, and this time, I fear there's..."


In [13]:
# Filling in missing values 
philippines.lyrics[0] = """Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Shout-out to your mom and dad for makin' you
Standin' ovation, they did a great job raisin' you
When I create, you're my muse
That kind of smile that makes the news
Can't nobody throw shade on your name in these streets
Triple threat, you a boss, you a bae, you a beast
You make it easy to choose
You got a mean touch, I can't refuse (No, I can't refuse it)
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread
Heart full of equity, you're an asset
Make sure that you don't need no mentions
Yeah, these are my only intentions
Already pass, you don't need no approval
Good everywhere, don't worry 'bout no refusal
Second to none, you got the upper hand now
Don't need a sponsor, nope, you're the brand now
You're my rock, my Colorado
Get that ring, just like Toronto
Love you now, a little more tomorrow
This how I feel, act like you know that you are
Picture perfect, you don't need no filter
Gorgeous, make 'em drop dead, you a killer
Shower you with all my attention
Yeah, these are my only intentions
Stay in the kitchen cookin' up, got your own bread (Whip it)
Heart full of equity, you're an asset (Asset)
Make sure that you don't need no mentions (Yeah, yeah)
Yeah, these are my only intentions (Quavo)
No cap, no pretendin', you don't need mentions (No cap)
Got 'em sayin' "goals," they don't wanna be independent ('Pendent)
Tell them to mind your business (Woo), we in our feelings
It's fifty-fifty percentage (Fifty), attention, we need commitment (Oh)
We gotta both admit it (Both), it's funny, we both listen (Both)
It's a blessing (Blessing) 'cause we both get it (Both)
You the best thing (Woo), and I don't need a witness (Best thing)
I'ma find me a ring and pray it's perfect fitted (Perfect, perfect)
Picture perfect, you don't need no filter (No filter)
Gorgeous, make 'em drop dead, you a killer (Oh-oh)
Shower you with all my attention (I will)
Yeah, these are my only intentions (Yeah)
Stay in the kitchen cookin' up, got your own bread (You do)
Heart full of equity, you're an asset (Uh-huh)
Make sure that you don't need no mentions (No mentions)
Yeah, these are my only intentions
Only intentions
That's all I plan to do"""

philippines.lyrics[43] = """You gave me a shoulder when I needed it
You showed me love when I wasn't feeling it
You helped me fight when I was giving in
And you made me laugh when I was losing it
'Cause you are, you are
The reason why I'm still hanging on
'Cause you are, you are
The reason why my head is still above water
And if I could, I'd get you the moon
And give it to you
And if death was coming for you
I'd give my life for you
'Cause you are, you are
The reason why I'm still hanging on
'Cause you are, you are
The reason why my head is still above water
And if I could, I'd get you the moon
And give it to you
And if death was coming for you
I'd give my life for you
'Cause you are, you are
Oh, you are
Oh, you are
You are
'Cause you are, you are
The reason why I'm still hanging on
'Cause you are, you are
The reason why my head is still above water
And if I could, I'd get you the moon
And give it to you
And if death was coming for you
I'd give my life for you"""

philippines.lyrics[48] = """Pabagalin muna natin ang ikot ng mundo
Pahintuin mga kamay ng oras sa relo
Dahan dahan
Dahan dahan lang
Dahan dahan
Dahan dahan lang
Sobrang saya dahil
Nandito ka na kahit
Mamaya lang
Ay aalis ka na rin agad
Wag ka sanang mainis
Panahon nati'y mabilis
Sandal mo muna sandali
Di naman nagmamadali, di'ba?
Dantay ka muna sakin beh
Sulitin natin ang gabi
Iyong mga problema'y itabi
Labi mo saakin idampi at
Kahit anong lalim
Aking sisisirin
Hindi na mabibitin
Pa sa aking gagawin
Pabagalin muna natin ang ikot ng mundo
(Ako'y sasayaw sa himig mo atin ang gabi na 'to)
Pahintuin mga kamay ng oras sa relo
(Takasan natin ang gulo, lilipad papalayo)
Dahan dahan
Dahan dahan lang
Dahan dahan
Dahan dahan lang
Dahan dahan ka lang
Ngayon alam ko namang
Hanggang dito ka lang
Kalaban ang orasan
Pano na 'ko pag gabi?
Di sanay nang walang katabi
Sandal mo muna sandali
Di naman nagmamadali, kaya
Pabagalin muna natin ang ikot ng mundo
(Ako'y sasayaw sa himig mo atin ang gabi na 'to)
Pahintuin mga kamay ng oras sa relo
(Takasan natin ang gulo, lilipad papalayo)
Dahan dahan
Dahan dahan lang
Dahan dahan
Dahan dahan lang
Ohhh pwede bang magpahinga muna saglit
Dahan dahan lang
Ohhh pwede bang magpahinga muna saglit
Dahan dahan lang"""

In [14]:
# Cleaning the Philippines lyrics 
philippines.lyrics = philippines.lyrics.apply(lambda x: clean_lyrics(x))

In [ ]:
# Writing the lyrics to file 
write_to_file(philippines)

In [15]:
# Importing the Netherlands dataset 
netherlands_path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\30.03.2020\Netherlands Top 50.csv'
pwd = os.getcwd()
os.chdir(os.path.dirname(netherlands_path))
netherlands = pd.read_csv(os.path.basename(netherlands_path), encoding="ISO-8859-1", index_col=0)
os.chdir(pwd)

In [16]:
# Inspecting the dataset
netherlands = netherlands[['song_title', 'artist/s', 'country']]
netherlands['lyrics'] = netherlands['song_title']
netherlands.head()

,song_title,artist/s,country,lyrics
0,17 Miljoen Mensen - Live @538 in Ahoy,"['Davina Michelle', 'Snelle']",Netherlands,17 Miljoen Mensen - Live @538 in Ahoy
1,Blinding Lights,The Weeknd,Netherlands,Blinding Lights
2,Jongen Van De Straat,Lil Kleine,Netherlands,Jongen Van De Straat
3,Anderhalf (feat. Poke & Judeska),"['Ali B', 'Poke', 'Judeska']",Netherlands,Anderhalf (feat. Poke & Judeska)
4,Roses - Imanbek Remix,"['SAINt JHN', 'Imanbek']",Netherlands,Roses - Imanbek Remix


In [17]:
# Getting lyrics for the dataset 
get_lyrics_reboot(netherlands)

,song_title,artist/s,country,lyrics
0,17 Miljoen Mensen - Live @538 in Ahoy,"['Davina Michelle', 'Snelle']",Netherlands,NaN
1,Blinding Lights,The Weeknd,Netherlands,Yeah\n\nI've been tryna call\nI've been on my ...
2,Jongen Van De Straat,Lil Kleine,Netherlands,"Je bent al vaker weggelopen\nSoms terecht, maa..."
3,Anderhalf (feat. Poke & Judeska),"['Ali B', 'Poke', 'Judeska']",Netherlands,"Unleaded\nMB\n\nAnderhalf meter (Afstand), and..."
4,Roses - Imanbek Remix,"['SAINt JHN', 'Imanbek']",Netherlands,Roses\nI walked in the corner with the body sc...
5,Don't Start Now,Dua Lipa,Netherlands,"If you don't wanna see me\n\nDid a full 180, c..."
6,Loop Niet Weg,"['Kris Kross Amsterdam', 'Tino Martin', 'Emma ...",Netherlands,"Songtekst van Kris Kross Amsterdam, Tino Marti..."
7,Smoorverliefd,Snelle,Netherlands,"Songtekst van Snelle - ""Smoorverliefd""\n\nÉén,..."
8,Het Is Al Laat Toch,Racoon,Netherlands,"Wat een idee, wat een plan, oh verdomme man\nW..."
9,Intentions (feat. Quavo),"['Justin Bieber', 'Quavo']",Netherlands,"Imagen perfecta, no necesitas filtro\nMagnífic..."


In [28]:
# Filling in missing values 
netherlands.lyrics[0] = """Ik zou eigenlijk juist nu
Een arm om je heen willen slaan
Zoveel nieuws, zo stil is het op straat
Een elleboog was nooit zo beschaafd
En er is wel meer raars want
Ik bel m'n moeder met een trilling in de stem
Door wat er moest van de minister-president
Goed bedoeld, stiekem best een beetje eng
En ik denk aan
Alle zorg om gezondheid en banen
Op televisie is nu alles zo beladen
Ik denk aan grootmoeders en vaders
Maar gek genoeg brengt het ons samen
Zeventien miljoen mensen
Op dat hele kleine stukje aarde
Die schrijf je niet de wetten voor
Die laat je in hun waarde
Zo zitten d'r nu duizenden studenten
In de lente op hun kamer
En ondertussen kneiterharde werkers
In de zorg, die amper slapen, maar met
Zeventien miljoen mensen
Is het best wel even schrikken
Zit de helft inmiddels thuis, maar met
Zeventien miljoen mensen
Met de neus in dezelfde richting komen we hier uit, met
Zeventien miljoen mensen
Op dat hele kleine stukje aarde
Die schrijf je niet de wetten voor
Die laat je in hun waarde"""

netherlands.lyrics[13] = """A veces tomo pa' poder desahogarme
Te lo confieso antes de que sea muy tarde
Sé que te perdí y nunca entendí por qué
Als alle keren dat m'n dromen over jou uit zouden komen
Was je elke dag en nacht bij mij
Ik wil niet denken aan je handen en je lippen bij een ander
Maar ik weet het is de werkelijkheid
Por eso tomo pa' olvidarte
Porque sinceramente duele recordarte
Si tú no estás la soledad ganó el combate
La luna no sale si no te vuelvo a ver
Daarom drink ik om jou te vergeten
Ik heb al dagen niet geslapen of gegeten
Ik ben al lang onder de eenzaamheid bezweken
De maan komt niet op als jij er niet meer bent
Vamos tomanos cuatro shots más de tequila
Pa que se dilaten las pupilas
Yo quiero verte bien todito el cuerpo
Que esta noche que hay enterer un muerto
Ik weet dat ik er niet altijd voor je kon zijn
Maar schatje zonder jou ga ik dood van de pijn
Want toen ik je zag, op die eerste dag, was ik zo van slag
Oh, ik wil je in m'n armen maar ik weet niet of dat mag van jou
Por eso tomo pa' olvidarte
Porque sinceramente duele recordarte
Si tú no estás la soledad ganó el combate
La luna no sale si no te vuelvo a ver
Daarom drink ik om jou te vergeten
Ik heb al dagen niet geslapen of gegeten
Ik ben al lang onder de eenzaamheid bezweken
De maan komt niet op als jij er niet meer bent
Baby, yo no tengo apuro
Vamos hacerlo de a poquito
Pegáte con disimulo
Que si hay miedo te lo quito
Ik geef toe dat ik het voor ons heb verkloot
Ik zocht te veel naar aandacht en ik was zo idioot
Dat het leek alsof ik jou niet meer zag staan
En nu ben je onbereikbaar en is alles misgegaan
Sinceridad
Tú te fuiste sin necesidad
Tú volviste sin necesidad
Pero tú te fuiste con tu ausencia y tu soledad
Ik weet dat ik er niet altijd voor je kon zijn
Ja, schatje zonder jou ga ik dood van de pijn
Want toen ik je zag, op die eerste dag, was ik zo van slag
Ik wil je in m'n armen maar ik weet niet of dat mag van jou
Voy a beber y sé
Que voy a enloquecer
Het liefste bel ik jou om te zeggen dat ik het allermeest van je hou
Por eso tomo pa' olvidarte
Porque sinceramente duele recordarte
Si tú no estás la soledad ganó el combate
La luna no sale si no te vuelvo a ver
Daarom drink ik om jou te vergeten
Ik heb al dagen niet geslapen of gegeten
Ik ben al lang onder de eenzaamheid bezweken
De maan komt niet op als jij er niet meer bent"""

netherlands.lyrics[20] = """Living in the fast lane
Tryna catch up with my own game
But the world is moving fast
I know that at the end of day
When the ceiling keeps the night away
I made it home at last, oh-oh
Give me stress, give me pressure
Make me crash and I'll come back faster
Beat me till I am first
Beat me and make me work
I'll show you what I'm worth
So beat me and make me first, hey
Oh-oh, oh-oh-oh-oh
Oh-oh-oh, oh-oh-oh-oh
Oh-oh, oh-oh-oh-oh
So beat me till I am first (Oh-oh-oh, oh-oh-oh-oh)
Ready, set, go
Shake me, wake me, make me
Pull up even faster
Tracing after everything that breaks me
Aches me, watch me pass it all
Love it when you shake me, quake me, make me
Pull up even faster
Tracing after everything that breaks me
I'ma race it all
Beat me till I am first
Beat me and make me work
I'll show you what I'm worth
So beat me and make me first
Oh-oh, oh-oh-oh-oh (Till I am first)
Oh-oh-oh, oh-oh-oh-oh (Till I am first)
Oh-oh, oh-oh-oh-oh
Oh-oh-oh, oh-oh-oh-oh (Beat me and make me first, hey)
I know that till the end of time
I will never meet the finish line
But I'll come home at last"""

In [30]:
# Cleaning the Netherlands lyrics
netherlands.lyrics = netherlands.lyrics.apply(lambda x: clean_lyrics(x))

In [741]:
# Writing the lyrics to file 
write_to_file(netherlands)

'Done!'

In [47]:
# Defining save .csv function
def save_csv(playlist_df): 
    path = r'D:\AllWomen\_Final Project\Spotify Project\Datasets\Lyrics'     
    os.chdir(path)
    file_name = '{} Lyrics Dataset.csv'.format(playlist_df.country[8]) 
    playlist_df.to_csv(file_name, encoding='ANSI')

In [48]:
# As well as saving the lyrics for each country to a .txt file, I saved all the datasets with lyrics to a new .csv
save_csv(usa)